In [ ]:
'''
Student Names + (IDs): 

Abdelrahman Mohamed Kamal Abdelaziz (1220255)
Mazen Ahmed Fouad Abdelwahab (1220269)
Mohamed Hesham Ibrahim Hassanain (1220278)
Ahmed Walaa Abdlelkhalek Abdelrahman (1220216)

'''
#Import(s)


import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import cv2
from skimage.feature import canny
from skimage.color import rgb2gray

In [ ]:
def show_images(images,titles=None):
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

In [ ]:
# Plate Localization

#-------------------------------------------------------------------------------------------------------------------------------

Path1 = 'Localization Test/One Step All Angles No Headlights/Straight.jpeg'
Path2 = 'Localization Test/Two Step All Angles No Headlights/Straight.jpeg'
Path3 = 'Localization Test/Three Step All Angles No Headlights/Straight.jpeg'
Path4 = 'Localization Test/One Step All Angles With Headlights/Straight.jpeg'
Path5 = 'Localization Test/Two Step All Angles With Headlights/Straight.jpeg'

Path11 = 'Localization Test/One Step All Angles No Headlights/Elevated.jpeg'
Path12 = 'Localization Test/Two Step All Angles No Headlights/Elevated.jpeg'
Path13 = 'Localization Test/Three Step All Angles No Headlights/Elevated.jpeg'
Path14 = 'Localization Test/One Step All Angles With Headlights/Elevated.jpeg'
Path15 = 'Localization Test/Two Step All Angles With Headlights/Elevated.jpeg'

Path111 = 'Localization Test/One Step All Angles No Headlights/Slight Right.jpeg'
Path112 = 'Localization Test/Two Step All Angles No Headlights/Slight Right.jpeg'
Path113 = 'Localization Test/Three Step All Angles No Headlights/Slight Right.jpeg'

Path1111 = 'Localization Test/One Step All Angles No Headlights/Slight Left.jpeg'
Path1112 = 'Localization Test/Two Step All Angles No Headlights/Slight Left.jpeg'
Path1113 = 'Localization Test/Three Step All Angles No Headlights/Slight Left.jpeg'
Path1114 = 'Localization Test/One Step All Angles With Headlights/Left.jpeg'

Path11111 = 'Localization Test/One Step All Angles No Headlights/Extreme Left.jpeg'
Path11112 = 'Localization Test/One Step All Angles No Headlights/Extreme Right.jpeg'
Path11113 = 'Localization Test/Two Step All Angles No Headlights/Extreme Left.jpeg'
Path11114 = 'Localization Test/Two Step All Angles No Headlights/Extreme Right.jpeg'
Path11115 = 'Localization Test/Three Step All Angles No Headlights/Extreme Left.jpeg'
Path11116 = 'Localization Test/Three Step All Angles No Headlights/Extreme Right.jpeg'
   
#-------------------------------------------------------------------------------------------------------------------------------

def Plate_Detection(image_path):

    # Original Image
    Original_Img = io.imread(image_path)[:,:,:3]
    Height, Width, _ = Original_Img.shape

    # Read Image and convert to grayscale
    img = cv2.imread(image_path)[:,:,:3]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply CLAHE to improve local contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray_clahe = clahe.apply(gray)

    # Apply Gaussian Blur to reduce noise
    gray_blur = cv2.GaussianBlur(gray_clahe, (5,5), 0)

    # Adaptive Thresholding to handle dark and bright regions
    thresh = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 19, 9)

    # Morphological closing to connect fragmented regions
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    plate_candidates = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / h
        area_ratio = (w * h) / (Height * Width)
        if (1.8 < aspect_ratio < 3.7) and (0.01 < area_ratio < 0.3):
            plate_candidates.append((x, y, w, h))

    if not plate_candidates:
        print("No plate candidate found.")
        return None

    # Choose the best candidate
    Best_Score = -1
    Best_Candidate = None

    for (x, y, w, h) in plate_candidates:

        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

        Current_Image = Original_Img[y:y+h, x:x+w]

        # --- Edge Density ---
        CannyEdges = canny(rgb2gray(Current_Image), sigma=1.0)
        edge_ratio = np.count_nonzero(CannyEdges) / (h * w)

        # --- Color / brightness variance ---
        # use brightness (mean over channels) as a simple measure
        brightness = Current_Image.mean(axis=2)
        brightness_var = np.var(brightness)

        # show_images([Current_Image], [f"Candidate Plate Edge Ratio: {edge_ratio:.4f} , Brightness Var: {brightness_var:.4f}"])

        # --- Combined score ---
        # Edge density is still the main factor, color helps down-weight bad ones
        score = brightness_var

        if score > Best_Score:
            Best_Score = score
            Best_Candidate = (x, y, w, h)


    # Crop the best candidate
    x, y, w, h = Best_Candidate
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # show_images([img], ["Best Plate Candidate"])
    plate_img = Original_Img[y:y+h, x:x+w] 
    
    return plate_img

#-------------------------------------------------------------------------------------------------------------------------------
# Testing the Plate Detection Function on Different Images
#-------------------------------------------------------------------------------------------------------------------------------

Origignal_Img1 = io.imread(Path1)
Detected_Plate1 = Plate_Detection(Path1)
if Detected_Plate1 is not None:
    show_images([Origignal_Img1, Detected_Plate1], ["One Step No Headlights", "Plate Image"])

Origignal_Img2 = io.imread(Path2)
Detected_Plate2 = Plate_Detection(Path2)
if Detected_Plate2 is not None:
    show_images([Origignal_Img2, Detected_Plate2], ["Two Steps No Headlights", "Plate Image"])

Origignal_Img3 = io.imread(Path3)
Detected_Plate3 = Plate_Detection(Path3)
if Detected_Plate3 is not None:
    show_images([Origignal_Img3, Detected_Plate3], ["Three Steps No Headlights", "Plate Image"])

Origignal_Img4 = io.imread(Path4)
Detected_Plate4 = Plate_Detection(Path4)
if Detected_Plate4 is not None:
    show_images([Origignal_Img4, Detected_Plate4], ["One Step Headlights", "Plate Image"])

Origignal_Img5 = io.imread(Path5)
Detected_Plate5 = Plate_Detection(Path5)
if Detected_Plate5 is not None:
    show_images([Origignal_Img5, Detected_Plate5], ["Two Steps Headlights", "Plate Image"])

#-------------------------------------------------------------------------------------------------------------------------------

Origignal_Img11 = io.imread(Path11)
Detected_Plate11 = Plate_Detection(Path11)
if Detected_Plate11 is not None:
    show_images([Origignal_Img11, Detected_Plate11], ["One Step Elevated No Headlights", "Plate Image"]) 

Origignal_Img12 = io.imread(Path12)
Detected_Plate12 = Plate_Detection(Path12)
if Detected_Plate12 is not None:
    show_images([Origignal_Img12, Detected_Plate12], ["Two Step Elevated No Headlights", "Plate Image"])

Origignal_Img13 = io.imread(Path13)
Detected_Plate13 = Plate_Detection(Path13)
if Detected_Plate13 is not None:
    show_images([Origignal_Img13, Detected_Plate13], ["Three Step Elevated No Headlights", "Plate Image"])

Origignal_Img14 = io.imread(Path14)
Detected_Plate14 = Plate_Detection(Path14) 
if Detected_Plate14 is not None:
    show_images([Origignal_Img14, Detected_Plate14], ["One Step Elevated Headlights", "Plate Image"])

Origignal_Img15 = io.imread(Path15)
Detected_Plate15 = Plate_Detection(Path15)  
if Detected_Plate15 is not None:
    show_images([Origignal_Img15, Detected_Plate15], ["Two Step Elevated Headlights", "Plate Image"])

#-------------------------------------------------------------------------------------------------------------------------------

Origignal_Img111 = io.imread(Path111)
Detected_Plate111 = Plate_Detection(Path111)
if Detected_Plate111 is not None:
    show_images([Origignal_Img111, Detected_Plate111], ["One Step Slight Right No Headlights", "Plate Image"])

Origignal_Img112 = io.imread(Path112)
Detected_Plate112 = Plate_Detection(Path112)
if Detected_Plate112 is not None:
    show_images([Origignal_Img112, Detected_Plate112], ["Two Step Slight Right No Headlights", "Plate Image"])

Origignal_Img113 = io.imread(Path113)
Detected_Plate113 = Plate_Detection(Path113)
if Detected_Plate113 is not None:
    show_images([Origignal_Img113, Detected_Plate113], ["Three Step Slight Right No Headlights", "Plate Image"])

# Origignal_Img114 = io.imread(Path114)
# Detected_Plate114 = Plate_Detection(Path114)
# if Detected_Plate114 is not None:
#     show_images([Origignal_Img114, Detected_Plate114], ["One Step Slight Right Headlights", "Plate Image"])

# Origignal_Img115 = io.imread(Path115)
# Detected_Plate115 = Plate_Detection(Path115)
# if Detected_Plate115 is not None:
#     show_images([Origignal_Img115, Detected_Plate115], ["Two Step Slight Right Headlights", "Plate Image"])

#-------------------------------------------------------------------------------------------------------------------------------

Origignal_Img1111 = io.imread(Path1111)
Detected_Plate1111 = Plate_Detection(Path1111)
if Detected_Plate1111 is not None:
    show_images([Origignal_Img1111, Detected_Plate1111], ["One Step Slight Left No Headlights", "Plate Image"])

Origignal_Img1112 = io.imread(Path1112)
Detected_Plate1112 = Plate_Detection(Path1112)
if Detected_Plate1112 is not None:
    show_images([Origignal_Img1112, Detected_Plate1112], ["Two Step Slight Left No Headlights", "Plate Image"])

Origignal_Img1113 = io.imread(Path1113)
Detected_Plate1113 = Plate_Detection(Path1113)
if Detected_Plate1113 is not None:
    show_images([Origignal_Img1113, Detected_Plate1113], ["Three Step Slight Left No Headlights", "Plate Image"])

Origignal_Img1114 = io.imread(Path1114)
Detected_Plate1114 = Plate_Detection(Path1114)
if Detected_Plate1114 is not None:
    show_images([Origignal_Img1114, Detected_Plate1114], ["One Step Slight Left Headlights", "Plate Image"])

# Origignal_Img1115 = io.imread(Path1115)
# Detected_Plate1115 = Plate_Detection(Path1115)
# if Detected_Plate1115 is not None:
#     show_images([Origignal_Img1115, Detected_Plate1115], ["Two Step Slight Left Headlights", "Plate Image"])

#-------------------------------------------------------------------------------------------------------------------------------

Original_Img11111 = io.imread(Path11111)
Detected_Plate11111 = Plate_Detection(Path11111)
if Detected_Plate11111 is not None:
    show_images([Original_Img11111, Detected_Plate11111], ["One Step Extreme Left No Headlights", "Plate Image"])

Original_Img11112 = io.imread(Path11112)
Detected_Plate11112 = Plate_Detection(Path11112)
if Detected_Plate11112 is not None:
    show_images([Original_Img11112, Detected_Plate11112], ["One Step Extreme Right No Headlights", "Plate Image"])

Original_Img11113 = io.imread(Path11113)
Detected_Plate11113 = Plate_Detection(Path11113)
if Detected_Plate11113 is not None:
    show_images([Original_Img11113, Detected_Plate11113], ["Two Step Extreme Left No Headlights", "Plate Image"])

Original_Img11114 = io.imread(Path11114)
Detected_Plate11114 = Plate_Detection(Path11114)
if Detected_Plate11114 is not None:
    show_images([Original_Img11114, Detected_Plate11114], ["Two Step Extreme Right No Headlights", "Plate Image"])

Original_Img11115 = io.imread(Path11115)
Detected_Plate11115 = Plate_Detection(Path11115)
if Detected_Plate11115 is not None:
    show_images([Original_Img11115, Detected_Plate11115], ["Three Step Extreme Left No Headlights", "Plate Image"])

Original_Img11116 = io.imread(Path11116)
Detected_Plate11116 = Plate_Detection(Path11116)
if Detected_Plate11116 is not None:
    show_images([Original_Img11116, Detected_Plate11116], ["Three Step Extreme Right No Headlights", "Plate Image"])

#-------------------------------------------------------------------------------------------------------------------------------

Original_Img11117 = io.imread('trella.png')[:,:,:3]
Detected_Plate11117 = Plate_Detection('trella.png')
if Detected_Plate11117 is not None:
    show_images([Original_Img11117, Detected_Plate11117], ["Trella", "Plate Image"])

# import os

# BASE_PATH = "Localization Test/Vehicles"

# START_ID = 1
# END_ID   = 2087

# for i in range(START_ID, END_ID + 1):
#     fname = f"{i:04d}.jpg"        # 0001.jpg, 0002.jpg, ...
#     fpath = os.path.join(BASE_PATH, fname)

#     if not os.path.exists(fpath):
#         print(f"[SKIP] Missing: {fpath}")
#         continue

#     try:
#         img = io.imread(fpath)    # RGB (as you want)
#         title = f"Vehicle_{fname}"
#         print(f"[RUN] {title}")
#         Detected_Plate11116=Plate_Detection(fpath)
#         if Detected_Plate11116 is not None:
#             show_images([img, Detected_Plate11116], ["img", "detect"])

#     except Exception as e:
#         print(f"[ERROR] {fname}: {e}")

In [ ]:
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np

def strip_corners_from_mask_edges(comp01, debug=False):
    """
    comp01: binary mask (H,W) of ONLY the strip component (0/1 or 0/255)
    Returns: (4,2) float32 corners in FULL image coords:
             [tl, tr, br, bl]  (already ordered)
    """

    comp = (comp01 > 0)
    H, W = comp.shape

    ys, xs = np.where(comp)
    if xs.size < 30:
        return None

    # x-span where the strip exists
    x_left  = int(np.percentile(xs, 2))
    x_right = int(np.percentile(xs, 98))
    if x_right <= x_left + 5:
        return None

    # For each x, find top and bottom y in that column
    top_pts = []
    bot_pts = []

    for x in range(x_left, x_right + 1):
        col = np.where(comp[:, x])[0]
        if col.size == 0:
            continue
        top_pts.append([x, float(col[0])])      # smallest y
        bot_pts.append([x, float(col[-1])])     # largest y

    top_pts = np.array(top_pts, dtype=np.float32)
    bot_pts = np.array(bot_pts, dtype=np.float32)

    if top_pts.shape[0] < 20 or bot_pts.shape[0] < 20:
        return None

    # robust trimming to ignore jagged ends / noise
    def _trim_points(pts, keep=0.80):
        n = pts.shape[0]
        k0 = int((1.0-keep)*0.5*n)
        k1 = int(n - k0)
        return pts[k0:k1]

    top_pts = _trim_points(top_pts, keep=0.80)
    bot_pts = _trim_points(bot_pts, keep=0.80)

    # Fit y = a*x + b using least squares (cv2.fitLine is OK too, but keep it simple)
    def _fit_line_y(ax_pts):
        x = ax_pts[:, 0]
        y = ax_pts[:, 1]
        A = np.stack([x, np.ones_like(x)], axis=1)
        a, b = np.linalg.lstsq(A, y, rcond=None)[0]
        return float(a), float(b)

    a_top, b_top = _fit_line_y(top_pts)
    a_bot, b_bot = _fit_line_y(bot_pts)

    # Evaluate at x_left/x_right
    tl = np.array([x_left,  a_top*x_left  + b_top], dtype=np.float32)
    tr = np.array([x_right, a_top*x_right + b_top], dtype=np.float32)
    bl = np.array([x_left,  a_bot*x_left  + b_bot], dtype=np.float32)
    br = np.array([x_right, a_bot*x_right + b_bot], dtype=np.float32)

    quad = np.array([tl, tr, br, bl], dtype=np.float32)

    # sanity: clamp y into image
    quad[:, 1] = np.clip(quad[:, 1], 0, H-1)

    # final order (safe)
    quad = _order_quad_tl_tr_br_bl(quad)

    if debug:
        area = abs(cv2.contourArea(quad))
        print(f"[mask-edge corners] xL={x_left} xR={x_right} area={area:.1f} "
              f"top: y={tl[1]:.1f}->{tr[1]:.1f}, bot: y={bl[1]:.1f}->{br[1]:.1f}")

    return quad


def cv2_getPerspectiveTransform_like(src_pts, dst_pts):
    """
    Drop-in replacement for cv2.getPerspectiveTransform(src, dst)
    where src and dst are (4,2).

    Returns:
        H (3,3) float64
    """
    src = np.asarray(src_pts, dtype=np.float64).reshape(4, 2)
    dst = np.asarray(dst_pts, dtype=np.float64).reshape(4, 2)

    # Solve for homography with h33 = 1
    A = np.zeros((8, 8), dtype=np.float64)
    b = np.zeros((8,), dtype=np.float64)

    for i in range(4):
        x, y = src[i]
        u, v = dst[i]

        # u = (h11*x + h12*y + h13) / (h31*x + h32*y + 1)
        A[2*i, 0] = x
        A[2*i, 1] = y
        A[2*i, 2] = 1
        A[2*i, 6] = -u * x
        A[2*i, 7] = -u * y
        b[2*i] = u

        # v = (h21*x + h22*y + h23) / (h31*x + h32*y + 1)
        A[2*i+1, 3] = x
        A[2*i+1, 4] = y
        A[2*i+1, 5] = 1
        A[2*i+1, 6] = -v * x
        A[2*i+1, 7] = -v * y
        b[2*i+1] = v

    p = np.linalg.solve(A, b)

    H = np.array([
        [p[0], p[1], p[2]],
        [p[3], p[4], p[5]],
        [p[6], p[7], 1.0]
    ], dtype=np.float64)

    return H


def _replicate_index(i, n):
    if i < 0:
        return 0
    if i >= n:
        return n - 1
    return i


def cv2_warpPerspective_like(img, H, dsize, flags=None, borderMode=None, borderValue=0):
    """
    Drop-in replacement for cv2.warpPerspective(img, H, dsize, flags=..., borderMode=...)

    Implements:
      - inverse mapping (like OpenCV default)
      - INTER_LINEAR
      - BORDER_REPLICATE

    Args:
      img: (H,W) or (H,W,C)
      H: (3,3) homography mapping src->dst (like cv2)
      dsize: (outW, outH)
      flags: ignored except assumed INTER_LINEAR
      borderMode: ignored except assumed BORDER_REPLICATE
    """
    src = np.asarray(img)
    outW, outH = int(dsize[0]), int(dsize[1])

    Hm = np.asarray(H, dtype=np.float64)
    Hinv = np.linalg.inv(Hm)  # OpenCV does inverse mapping internally

    inH, inW = src.shape[:2]
    hasC = (src.ndim == 3)
    C = src.shape[2] if hasC else 1

    # work in float for interpolation, then cast back
    src_f = src.astype(np.float64)

    if not hasC:
        out = np.empty((outH, outW), dtype=np.float64)
    else:
        out = np.empty((outH, outW, C), dtype=np.float64)

    # For speed: precompute row terms that depend on v
    for v in range(outH):
        # These are coefficients for u in numerator/denom
        a0 = Hinv[0, 0]
        a1 = Hinv[0, 1] * v + Hinv[0, 2]
        b0 = Hinv[1, 0]
        b1 = Hinv[1, 1] * v + Hinv[1, 2]
        c0 = Hinv[2, 0]
        c1 = Hinv[2, 1] * v + Hinv[2, 2]

        for u in range(outW):
            denom = c0 * u + c1
            if denom == 0:
                # replicate border -> clamp to nearest valid (choose 0,0)
                if not hasC:
                    out[v, u] = src_f[0, 0]
                else:
                    out[v, u, :] = src_f[0, 0, :]
                continue

            x = (a0 * u + a1) / denom
            y = (b0 * u + b1) / denom

            # bilinear sampling with BORDER_REPLICATE
            x0 = int(np.floor(x))
            y0 = int(np.floor(y))
            x1 = x0 + 1
            y1 = y0 + 1

            ax = x - x0
            ay = y - y0

            x0c = _replicate_index(x0, inW)
            x1c = _replicate_index(x1, inW)
            y0c = _replicate_index(y0, inH)
            y1c = _replicate_index(y1, inH)

            if not hasC:
                p00 = src_f[y0c, x0c]
                p10 = src_f[y0c, x1c]
                p01 = src_f[y1c, x0c]
                p11 = src_f[y1c, x1c]

                top = (1 - ax) * p00 + ax * p10
                bot = (1 - ax) * p01 + ax * p11
                out[v, u] = (1 - ay) * top + ay * bot
            else:
                p00 = src_f[y0c, x0c, :]
                p10 = src_f[y0c, x1c, :]
                p01 = src_f[y1c, x0c, :]
                p11 = src_f[y1c, x1c, :]

                top = (1 - ax) * p00 + ax * p10
                bot = (1 - ax) * p01 + ax * p11
                out[v, u, :] = (1 - ay) * top + ay * bot

    # cast back to original dtype like cv2
    if np.issubdtype(src.dtype, np.integer):
        info = np.iinfo(src.dtype)
        out = np.clip(out, info.min, info.max).astype(src.dtype)
    else:
        out = out.astype(src.dtype)

    return out


# ================================================================
#  DEBUG VIEW
# ================================================================
def _show_mask_debug(mask, title):
    plt.figure(figsize=(4, 2))
    plt.imshow(mask, cmap="gray")
    plt.title(title)
    plt.axis("off")
    plt.show()

# ======================================================================
#  HELPER: draw candidate rectangles on debug image
# ======================================================================
def draw_candidates_debug(img, candidates, best_idx):
    debug_img = img.copy()

    for i, c in enumerate(candidates):
        color = (0,255,0) if i==best_idx else \
                (random.randint(50,255), random.randint(50,255), random.randint(50,255))

        x0, y0, x1, y1 = c["x0"], c["y0"], c["x1"], c["y1"]
        cv2.rectangle(debug_img, (x0,y0), (x1,y1), color, 2)
        cv2.putText(debug_img, f"{c['score']:.2f}", (x0, y0-4),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    return debug_img

def _order_quad_tl_tr_br_bl(pts4):
    pts = np.array(pts4, dtype=np.float32)
    s = pts.sum(axis=1)              # x+y
    d = pts[:,0] - pts[:,1]          # x-y
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmax(d)]
    bl = pts[np.argmin(d)]
    return np.array([tl, tr, br, bl], dtype=np.float32)

def deskew_stage2B_strip_perspective_if_needed(
    img_rgb,
    strip_corners_xy,     # 4x2 corners IN img_rgb coords
    debug=False,
    trap_ratio_thr=0.08,  # how trapezoid before we apply
    min_strip_area=80.0   # ignore tiny / unstable quads
):
    """
    Stage 2B: Projective correction using ONLY the 4 corners of the blue strip.
    Returns dict with 'rgb' and 'did_apply'.
    """

    if strip_corners_xy is None:
        return {"rgb": img_rgb, "did_apply": False, "reason": "no_corners"}

    quad = np.array(strip_corners_xy, dtype=np.float32)
    if quad.shape != (4, 2):
        return {"rgb": img_rgb, "did_apply": False, "reason": "bad_shape"}

    quad = _order_quad_tl_tr_br_bl(quad)

    # quick sanity: area of quad
    area = float(abs(cv2.contourArea(quad)))
    if area < min_strip_area:
        return {"rgb": img_rgb, "did_apply": False, "reason": "tiny_quad"}

    tl, tr, br, bl = quad

    def _dist(a, b):
        return float(np.hypot(a[0]-b[0], a[1]-b[1]))

    top_w    = _dist(tl, tr)
    bot_w    = _dist(bl, br)
    left_h   = _dist(tl, bl)
    right_h  = _dist(tr, br)

    # trapezoid-ness measures (1.0 is perfect rectangle)
    w_ratio = (top_w + 1e-6) / (bot_w + 1e-6)
    h_ratio = (left_h + 1e-6) / (right_h + 1e-6)

    need = (abs(w_ratio - 1.0) > trap_ratio_thr) or (abs(h_ratio - 1.0) > trap_ratio_thr)

    if debug:
        print(f"[2B] area={area:.1f} top/bot={w_ratio:.3f} left/right={h_ratio:.3f} -> apply={need}")

    if not need:
        return {"rgb": img_rgb, "did_apply": False, "reason": "already_rect"}

    # target rectangle size based on measured strip size
    dst_w = int(max(top_w, bot_w))
    dst_h = int(max(left_h, right_h))
    dst_w = max(dst_w, 10)
    dst_h = max(dst_h, 6)

    dst = np.array([
        [0,      0],
        [dst_w-1,0],
        [dst_w-1,dst_h-1],
        [0,      dst_h-1]
    ], dtype=np.float32)
    Himg, Wimg = img_rgb.shape[:2]
    Hm = cv2_getPerspectiveTransform_like(quad, dst)
    warped = cv2_warpPerspective_like(img_rgb, Hm, (Wimg, Himg))   #,flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REPLICATE    HERE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    return {"rgb": warped, "did_apply": True, "reason": "warped"}



# ================================================================
#  TIERS
# ================================================================
BLUE_PARAMS_STRICT = dict(BLUE_SAT_THRESH=140, BLUE_VAL_THRESH=110, BLUE_HUE_MIN=80, BLUE_HUE_MAX=120, BLUE_DELTA=12)
BLUE_PARAMS_MID    = dict(BLUE_SAT_THRESH=115, BLUE_VAL_THRESH=95,  BLUE_HUE_MIN=80, BLUE_HUE_MAX=120, BLUE_DELTA=14)
BLUE_PARAMS_LENIENT= dict(BLUE_SAT_THRESH=60,  BLUE_VAL_THRESH=80,  BLUE_HUE_MIN=75, BLUE_HUE_MAX=125, BLUE_DELTA=18)
BLUE_PARAMS_VERY_LENIENT = dict(BLUE_SAT_THRESH=30, BLUE_VAL_THRESH=60, BLUE_HUE_MIN=70, BLUE_HUE_MAX=135, BLUE_DELTA=22)

BLUE_PARAMS_ULTRA_LENIENT = dict(
    BLUE_SAT_THRESH=18,
    BLUE_VAL_THRESH=45,
    BLUE_HUE_MIN=65,
    BLUE_HUE_MAX=145,
    BLUE_DELTA=26
)

BLUE_PARAMS_MAX_LENIENT = dict(
    BLUE_SAT_THRESH=8,
    BLUE_VAL_THRESH=35,
    BLUE_HUE_MIN=55,
    BLUE_HUE_MAX=160,
    BLUE_DELTA=32
)

BLUE_PARAMS_NUCLEAR = dict(
    BLUE_SAT_THRESH = 3,
    BLUE_VAL_THRESH = 25,
    BLUE_HUE_MIN    = 45,
    BLUE_HUE_MAX    = 170,
    BLUE_DELTA      = 40
)

BLUE_PARAM_TIERS = [
    ("STRICT",       BLUE_PARAMS_STRICT),
    ("MID",          BLUE_PARAMS_MID),
    ("LENIENT",      BLUE_PARAMS_LENIENT),
    ("VERY_LENIENT", BLUE_PARAMS_VERY_LENIENT),
    ("ULTRA_LENIENT", BLUE_PARAMS_ULTRA_LENIENT),
    ("MAX_LENIENT",   BLUE_PARAMS_MAX_LENIENT),
    ("NUCLEAR",       BLUE_PARAMS_NUCLEAR),
]

TIER_BONUS = {
    "STRICT": 0.18,
    "MID": 0.12,
    "LENIENT": 0.06,
    "VERY_LENIENT": 0.00,
    "ULTRA_LENIENT": -0.06,
    "MAX_LENIENT":   -0.10,
    "NUCLEAR":       -0.10,
}

# ================================================================
#  SCORING HYPERPARAMS
# ================================================================
HUE_CENTER = 100
HUE_SIGMA  = 18.0

WHITE_S_MAX = 90
WHITE_V_MIN = 120

W_CC_DOM   = 0.22
W_ROW_BAND = 0.18
W_BLUE     = 0.26
W_SAT      = 0.10
W_BELOW    = 0.18
W_TOP_PRIOR= 0.06


# ================================================================
def _adaptive_blue_mask_with_params(img_rgb, params, debug=False, tag=""):
    hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    H, S, V = cv2.split(hsv)

    sat_thr = params["BLUE_SAT_THRESH"]
    val_thr = params["BLUE_VAL_THRESH"]
    hue_min = params["BLUE_HUE_MIN"]
    hue_max = params["BLUE_HUE_MAX"]
    delta   = params["BLUE_DELTA"]

    colored = (S > sat_thr) & (V > val_thr)
    H_colored = H[colored]

    hue_mask = (H_colored >= hue_min) & (H_colored <= hue_max)
    blue_hues = H_colored[hue_mask]

    if blue_hues.size == 0:
        if debug:
            print(f"[{tag}] No blue hues found in candidate window.")
        return np.zeros_like(H, dtype=np.uint8), None, None

    hist, bin_edges = np.histogram(blue_hues, bins=36, range=(0, 180))
    peak_bin = int(np.argmax(hist))
    h_peak = 0.5 * (bin_edges[peak_bin] + bin_edges[peak_bin + 1])

    h_low  = max(0,   int(h_peak - delta))
    h_high = min(179, int(h_peak + delta))

    lower_blue = np.array([h_low,  sat_thr, val_thr], dtype=np.uint8)
    upper_blue = np.array([h_high, 255,     255],     dtype=np.uint8)

    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    if debug:
        n = int(mask.sum() // 255)
        print(f"[{tag}] Peak={h_peak:.1f}, H=({h_low},{h_high}), S>{sat_thr}, V>{val_thr}, pixels={n}")

    return mask, lower_blue, upper_blue


def _hue_distance_circular(h, center):
    d = np.abs(h.astype(np.float32) - float(center))
    return np.minimum(d, 180.0 - d)


def _score_component(mask, hsv, comp_mask, bbox, tier_name, debug=False):
    Himg, Wimg = mask.shape[:2]
    x, y, w, h = bbox
    area = float(comp_mask.sum())
    total = float((mask > 0).sum()) + 1e-6

    cc_dom = area / total

    y_center = y + 0.5*h
    top_prior = 1.0 - np.clip(y_center / (0.65 * Himg + 1e-6), 0.0, 1.0)

    sub = mask[y:y+h, x:x+w] > 0
    if sub.size == 0:
        return -1e9, {}
    row_counts = sub.sum(axis=1).astype(np.float32)
    if row_counts.max() <= 0:
        row_band = 0.0
    else:
        peak = row_counts.max()
        mean = row_counts.mean() + 1e-6
        peakiness = float(peak / mean)
        thr = 0.6 * peak
        strong = row_counts >= thr
        longest = 0
        cur = 0
        for v in strong:
            if v:
                cur += 1
                longest = max(longest, cur)
            else:
                cur = 0
        run_frac = float(longest / (h + 1e-6))
        row_band = np.tanh((peakiness - 1.5) / 2.0) * 0.6 + run_frac * 0.4
        row_band = float(np.clip(row_band, 0.0, 1.0))

    Hc = hsv[...,0][comp_mask]
    Sc = hsv[...,1][comp_mask]
    if Hc.size == 0:
        blue_score = 0.0
        sat_score  = 0.0
    else:
        d = _hue_distance_circular(Hc, HUE_CENTER)
        mean_d = float(d.mean())
        blue_score = float(np.exp(- (mean_d**2) / (2.0 * (HUE_SIGMA**2))))
        sat_score = float(np.clip(Sc.mean() / 255.0, 0.0, 1.0))

    y0 = int(y + h)
    y1 = int(min(Himg, y + h + 1.5*h))
    if y1 <= y0 + 2:
        below_white = 0.0
    else:
        below = hsv[y0:y1, x:x+w]
        S_b = below[...,1].astype(np.uint8)
        V_b = below[...,2].astype(np.uint8)
        white = (S_b < WHITE_S_MAX) & (V_b > WHITE_V_MIN)
        below_white = float(white.mean())

    tier_bonus = TIER_BONUS.get(tier_name, 0.0)

    score = (
        W_CC_DOM    * cc_dom +
        W_ROW_BAND  * row_band +
        W_BLUE      * blue_score +
        W_SAT       * sat_score +
        W_BELOW     * below_white +
        W_TOP_PRIOR * top_prior +
        tier_bonus
    )

    if debug:
        print(f"    [{tier_name}] bbox={x,y,w,h} "
              f"score={score:.3f} | cc={cc_dom:.2f} row={row_band:.2f} blue={blue_score:.2f} "
              f"sat={sat_score:.2f} belowW={below_white:.2f} top={top_prior:.2f} +bonus={tier_bonus:.2f}")

    return float(score), dict(bbox=(x,y,w,h))


def adaptive_blue_mask(img_rgb, debug=False):
    # NOTE: keep everything minimal
    Himg, Wimg = img_rgb.shape[:2]

    # ------------------------------------------------------------
    # NEW: probe STRICT + MID, and if both are "near empty", rescue
    # ------------------------------------------------------------
    RESCUE_WHITE_THR = 200  # you can tune this
    did_rescue = False

    # quick probe on ORIGINAL img
    maskS, _, _ = _adaptive_blue_mask_with_params(img_rgb, BLUE_PARAMS_STRICT, debug=False, tag="PROBE_STRICT")
    maskM, _, _ = _adaptive_blue_mask_with_params(img_rgb, BLUE_PARAMS_MID,    debug=False, tag="PROBE_MID")

    nS = int(maskS.sum() // 255) if maskS is not None else 0
    nM = int(maskM.sum() // 255) if maskM is not None else 0

    img_work = img_rgb

    if (nS < RESCUE_WHITE_THR) and (nM < RESCUE_WHITE_THR):
        did_rescue = True
        if debug:
            print(f"[RESCUE TRIGGER] STRICT={nS} MID={nM} < {RESCUE_WHITE_THR} -> calling rescue_blue_strip_rgb_extreme()")

        img_rescued = rescue_blue_strip_rgb_extreme(img_rgb, debug=True)

        if debug:
            plt.figure(figsize=(10,4))
            plt.subplot(1,2,1); plt.imshow(img_rgb);     plt.title("Original (before rescue)"); plt.axis("off")
            plt.subplot(1,2,2); plt.imshow(img_rescued); plt.title("After rescue_blue_strip_rgb_extreme"); plt.axis("off")
            plt.tight_layout()
            plt.show()

        img_work = img_rescued

    # now proceed normally, BUT using img_work (original or rescued)
    hsv = cv2.cvtColor(img_work, cv2.COLOR_RGB2HSV)

    best = dict(
        score=-1e9,
        mask=None, lb=None, ub=None, tier=None, bbox=None,
        did_rescue=did_rescue,
        best_comp_id=None,
        best_labels=None
    )

    for tier_name, params in BLUE_PARAM_TIERS:
        mask, lb, ub = _adaptive_blue_mask_with_params(img_work, params, debug=debug, tag=tier_name)

        # keep your dilation exactly as-is
        mask = cv2.dilate(mask, cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3)), iterations=1)

        if debug:
            _show_mask_debug(mask, f"{tier_name} mask (raw+dilated)")

        if lb is None or mask.sum() == 0:
            continue

        bin_mask = (mask > 0).astype(np.uint8)
        num, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_mask, connectivity=8)

        for comp_id in range(1, num):
            x, y, w, h, area = stats[comp_id]
            if area < 20:
                continue

            comp_mask = (labels == comp_id)
            score, details = _score_component(mask, hsv, comp_mask, (x,y,w,h), tier_name, debug=debug)

            if score > best["score"]:
                best.update(
                    score=score,
                    mask=mask, lb=lb, ub=ub, tier=tier_name, bbox=(x,y,w,h),
                    best_comp_id=comp_id,
                    best_labels=labels
                )

    if best["mask"] is None:
        return np.zeros((Himg, Wimg), dtype=np.uint8), None, None, None, None, None, img_work

    # -----------------------------
    # NEW: compute 4 rotated corners of BEST strip component
    # -----------------------------
# -----------------------------
# NEW: compute 4 corners of BEST strip component
# Try TRUE quad from contour; fallback to minAreaRect rectangle
# -----------------------------



    # strip_corners = None
    # strip_corners_src = "none"

    # if best["best_labels"] is not None and best["best_comp_id"] is not None:
    #     comp = (best["best_labels"] == best["best_comp_id"]).astype(np.uint8) * 255

    #     contours, _ = cv2.findContours(comp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #     if contours:
    #         cnt = max(contours, key=cv2.contourArea)
    #         if cv2.contourArea(cnt) >= 30:

    #             # try to approximate to 4-point polygon (real trapezoid if perspective exists)
    #             peri = cv2.arcLength(cnt, True)
    #             approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)

    #             if len(approx) == 4:
    #                 strip_corners = approx.reshape(4, 2).astype(np.float32)
    #                 strip_corners_src = "approx4"
    #             else:
    #                 # fallback: minAreaRect gives rectangle (always w_ratio≈1, h_ratio≈1)
    #                 rect = cv2.minAreaRect(cnt)
    #                 strip_corners = cv2.boxPoints(rect).astype(np.float32)
    #                 strip_corners_src = f"minAreaRect(len={len(approx)})"



    strip_corners = None
    strip_corners_src = "none"

    if best["best_labels"] is not None and best["best_comp_id"] is not None:
        comp = (best["best_labels"] == best["best_comp_id"]).astype(np.uint8) * 255

        # NEW PRIMARY METHOD: mask edge lines -> trapezoid corners
        strip_corners = strip_corners_from_mask_edges(comp, debug=debug)
        if strip_corners is not None:
            strip_corners_src = "mask_edges"

        else:
            # fallback 1: approxPolyDP if it returns 4 points
            contours, _ = cv2.findContours(comp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                cnt = max(contours, key=cv2.contourArea)
                peri = cv2.arcLength(cnt, True)
                approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)

                if len(approx) == 4:
                    strip_corners = approx.reshape(4, 2).astype(np.float32)
                    strip_corners_src = "approx4"
                else:
                    # last fallback: minAreaRect (rectangle)
                    rect = cv2.minAreaRect(cnt)
                    strip_corners = cv2.boxPoints(rect).astype(np.float32)
                    strip_corners_src = f"minAreaRect(len={len(approx)})"



    if debug:
        x,y,w,h = best["bbox"]
        print(f"\n✅ CHOSEN OVERALL: tier={best['tier']} score={best['score']:.3f} bbox={(x,y,w,h)} rescue_used={did_rescue}\n")
        if strip_corners is not None:
            print(f"[STRIP CORNERS] src={strip_corners_src}\n", strip_corners)


    return best["mask"], best["lb"], best["ub"], best["tier"], best["bbox"], strip_corners, img_work



#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

def crop_left_right_by_blue_adaptive(img_bgr, debug=False, enable_perspective=True):

    H, W = img_bgr.shape[:2]


    mask, lb, ub, tier, bbox, strip_corners, img_work = adaptive_blue_mask(img_bgr, False)   #HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    if debug:
        print("✅ Tier used:", tier)

    if lb is None:
        if debug:
            print("❌ No blue detected")
        return None, None, mask, None

    # IMPORTANT: continue on img_work (original or rescued)
    hsv = cv2.cvtColor(img_work, cv2.COLOR_RGB2HSV)

    row_counts = mask.sum(axis=1)//255
    max_row = row_counts.max()
    if max_row == 0:
        return None, None, mask, None

    row_thresh = 0.30*max_row
    blue_rows = row_counts > row_thresh

    bands = []
    cur_start = None
    for y, flag in enumerate(blue_rows):
        if flag and cur_start is None:
            cur_start = y
        elif not flag and cur_start is not None:
            bands.append((cur_start, y))
            cur_start = None
    if cur_start is not None:
        bands.append((cur_start, len(blue_rows)))

    # =====================================================
    # ✅ NEW: two-pass gating thresholds (only used for filtering)
    # =====================================================
    PASS1_MIN_BAND_H_FRAC = 0.02
    PASS1_MIN_BAND_W_FRAC = 0.10

    PASS2_MIN_BAND_H_FRAC = 0.008
    PASS2_MIN_BAND_W_FRAC = 0.05

    def _collect_candidates(min_band_h_frac, min_band_w_frac):
        candidates = []

        for (y0_band, y1_band) in bands:
            band_h = y1_band - y0_band
            if band_h < min_band_h_frac * H:
                continue

            submask = mask[y0_band:y1_band, :]

            ys2, xs2 = np.where(submask > 0)
            if xs2.size == 0:
                continue

            x0_band = int(np.percentile(xs2, 1))
            x1_band = int(np.percentile(xs2, 99))
            band_w  = x1_band - x0_band

            if band_w < min_band_w_frac * W or band_w > 0.95 * W:
                continue

            ar_band = band_w / (band_h + 1e-6)
            s_ar_band = np.tanh((ar_band - 2.0) / 2.0)
            s_ar_band = float(np.clip(s_ar_band, 0.0, 1.0))

            region_hsv = hsv[:, x0_band:x1_band]
            _, Sc, Vc = cv2.split(region_hsv)

            S_MAX = 130
            V_MIN = 80
            white_mask = (Sc < S_MAX) & (Vc > V_MIN)
            white_ratio = float(white_mask.mean())

            s_white = white_ratio

            k = 3.0
            y0_pl = max(0,   int(y0_band - 0.2*band_h))
            y1_pl = min(H-1, int(y0_band + k*band_h))
            plate_h = y1_pl - y0_pl
            ar_plate = band_w / (plate_h + 1e-6)

            expected_ar = 2.0
            sigma = 0.8
            s_ar_plate = float(np.exp(-((ar_plate - expected_ar)**2) / (2 * sigma**2)))

            score = (
                0.20*s_ar_band +
                0.50*s_white   +
                0.30*s_ar_plate
            )

            candidates.append({
                "x0": x0_band, "y0": y0_band,
                "x1": x1_band, "y1": y1_band,
                "width": band_w,
                "height": band_h,
                "score": float(score),
                "white_ratio": white_ratio,
                "s_white": s_white,
                "s_ar_band": s_ar_band,
                "s_ar_plate": s_ar_plate
            })

        return candidates

    candidates = _collect_candidates(PASS1_MIN_BAND_H_FRAC, PASS1_MIN_BAND_W_FRAC)

    if not candidates:
        candidates = _collect_candidates(PASS2_MIN_BAND_H_FRAC, PASS2_MIN_BAND_W_FRAC)
        if debug and candidates:
            print("[2-PASS] Pass1 empty -> using lenient gates for far/small plate case")

    if not candidates:
        return img_work, (0,0,W-1,H-1), mask, img_work

    candidates_sorted = sorted(
        candidates,
        key=lambda c: (c["score"], c["white_ratio"], c["width"], c["height"]),
        reverse=True
    )

    best = candidates_sorted[0]
    best_idx = candidates.index(best)

    margin = int(0.02*W)
    x0 = max(0, best["x0"] - margin)
    x1 = min(W-1, best["x1"] + margin)

    y_strip_top = int(best["y0"])
    strip_h     = int(best["height"])

    plate_h = int(3.5 * strip_h)

    y_top = max(0, y_strip_top)
    y_bottom = y_top + plate_h + int(0.10 * strip_h)
    y_bottom = min(H - 1, y_bottom)

        # =====================================================
    # ✅ NEW: tiny safety padding (top/left/right) so we don't cut plate edges                             MAW OK OR NOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
    # =====================================================
    # PAD_X_FRAC = 0.010   # ~1% of image width
    # PAD_TOP_FRAC = 0.010 # ~1% of image height
    # PAD_X_PX_MIN = 2
    # PAD_TOP_PX_MIN = 2

    # pad_x  = max(PAD_X_PX_MIN,  int(PAD_X_FRAC  * W))
    # # pad_ty = max(PAD_TOP_PX_MIN, int(PAD_TOP_FRAC * H))
    # pad_ty = max(2, int(0.15 * strip_h))  # e.g., 15% of strip height


    # # expand crop slightly
    # x0 = max(0, x0 - pad_x)     # left
    # x1 = min(W - 1, x1 + pad_x) # right
    # y_top = max(0, y_top - pad_ty)  # top only (keep bottom as-is)

    # # (optional) keep bottom inside bounds (already clamped, but safe)
    # y_bottom = min(H - 1, y_bottom)


    cropped = img_work[y_top:y_bottom+1, x0:x1]


    # ===================== DEBUG: Stage 2B inspection =====================
    if debug:
        print("\n================= [DEBUG 2B] =================")
        print("[DEBUG 2B] img_work used instead of original:", (img_work is not img_bgr))
        print("[DEBUG 2B] crop x0,x1,y_top,y_bottom:", x0, x1, y_top, y_bottom)
        print("[DEBUG 2B] cropped shape:", cropped.shape)

        if strip_corners is None:
            print("[DEBUG 2B] strip_corners = None  (no corners computed!)")
        else:
            print("[DEBUG 2B] strip_corners (FULL coords):\n", strip_corners)

            # convert to cropped coords
            strip_corners_crop = strip_corners.copy().astype(np.float32)
            strip_corners_crop[:, 0] -= float(x0)
            strip_corners_crop[:, 1] -= float(y_top)

            print("[DEBUG 2B] strip_corners (CROPPED coords):\n", strip_corners_crop)

            # order and compute trapezoid measures like Stage 2B
            quad = _order_quad_tl_tr_br_bl(strip_corners_crop)
            tl, tr, br, bl = quad

            def _dist(a, b):
                return float(np.hypot(a[0]-b[0], a[1]-b[1]))

            top_w  = _dist(tl, tr)
            bot_w  = _dist(bl, br)
            left_h = _dist(tl, bl)
            right_h= _dist(tr, br)

            w_ratio = (top_w + 1e-6) / (bot_w + 1e-6)
            h_ratio = (left_h + 1e-6) / (right_h + 1e-6)


                        # =====================================================
            # ✅ NEW: cap extreme ratios (if too crazy -> skip perspective)
            # =====================================================
            MAX_RATIO_CAP = 2.0  # your requested cap

            w_ratio_sym = max(w_ratio, 1.0 / (w_ratio + 1e-6))   # symmetric ratio >= 1
            h_ratio_sym = max(h_ratio, 1.0 / (h_ratio + 1e-6))   # symmetric ratio >= 1

            if (w_ratio_sym > MAX_RATIO_CAP) or (h_ratio_sym > MAX_RATIO_CAP):
                if debug:
                    print(f"[2B] SKIP: extreme ratio (w_sym={w_ratio_sym:.3f}, h_sym={h_ratio_sym:.3f}) > {MAX_RATIO_CAP}")
                    enable_perspective=False


            area = float(abs(cv2.contourArea(quad)))
            print(f"[DEBUG 2B] quad area={area:.1f} top_w={top_w:.1f} bot_w={bot_w:.1f} w_ratio={w_ratio:.3f}")
            print(f"[DEBUG 2B] left_h={left_h:.1f} right_h={right_h:.1f} h_ratio={h_ratio:.3f}")

            # draw overlay on cropped image
            overlay = cropped.copy()
            qint = np.round(quad).astype(np.int32)
            cv2.polylines(overlay, [qint], True, (255, 0, 0), 2)  # blue polygon

            # label corners
            pts = {"TL": tl, "TR": tr, "BR": br, "BL": bl}
            for name, p in pts.items():
                px, py = int(round(p[0])), int(round(p[1]))
                cv2.circle(overlay, (px, py), 4, (0, 255, 0), -1)
                cv2.putText(overlay, name, (px + 5, py - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            plt.figure(figsize=(10,4))
            plt.subplot(1,2,1); plt.imshow(cropped);  plt.title("Cropped (before 2B)"); plt.axis("off")
            plt.subplot(1,2,2); plt.imshow(overlay);  plt.title("Cropped + strip corners"); plt.axis("off")
            plt.tight_layout()
            plt.show()

        print("================================================\n")
    # =================== END DEBUG: Stage 2B inspection ===================


    # =====================================================
    # ✅ NEW: Stage 2B plug-in (using strip corners only)
    # Convert strip_corners from FULL img_work coords -> CROPPED coords
    # =====================================================
    if enable_perspective and (strip_corners is not None):
        strip_corners_crop = strip_corners.copy()
        strip_corners_crop[:, 0] -= float(x0)
        strip_corners_crop[:, 1] -= float(y_top)

        out2b = deskew_stage2B_strip_perspective_if_needed(
            cropped, strip_corners_crop, debug=debug
        )
        if out2b["did_apply"]:
            cropped = out2b["rgb"]
            if debug:
                print("[2B] applied:", out2b.get("reason", ""))



    debug_img = draw_candidates_debug(img_work, candidates, best_idx)

    # if debug:
    #     print("\n========= CANDIDATE BREAKDOWN =========")
    #     for i, c in enumerate(candidates):
    #         print(f"[{i}] score={c['score']:.3f}, "
    #               f"white_ratio={c['white_ratio']:.3f}, "                                     HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #               f"s_white={c['s_white']:.2f}, "
    #               f"ARband={c['s_ar_band']:.2f}, "
    #               f"ARplate={c['s_ar_plate']:.2f}, "
    #               f"range=({c['x0']},{c['x1']}) "
    #               f"vertical=({c['y0']},{c['y1']})")
    #     print("Chosen:", best)
    #     print("========================================\n")

    return cropped, (best["x0"], best["y0"], best["x1"], best["y1"]), mask, debug_img




def debug_blue_crop(plate_img, title, enable_perspective=True):

    if plate_img is None:
        print(f"{title}: plate_img is None, skipping.")
        return None

    cropped, bbox, mask, debug_img = crop_left_right_by_blue_adaptive(
    plate_img, debug=True, enable_perspective=enable_perspective
)



    images = [plate_img]
    titles = [f"{title} - Original"]

    if mask is not None:
        images.append(mask)
        titles.append("Blue mask")

    if debug_img is not None:
        images.append(debug_img)
        titles.append("All candidates")
    else:
        print(f"{title}: debug_img is None (likely no candidates / no blue).")

    if cropped is not None:
        images.append(cropped)
        titles.append("Final crop")
    else:
        print(f"{title}: cropped is None (no blue detected).")

    show_images(images, titles)
    plt.close('all')

    return cropped


def rescue_blue_strip_rgb_extreme(img_rgb, debug=False):

    """
    MUCH harsher rescue:
      - normalize + CLAHE + stronger gamma on V
      - build a better seed (prefers blue-ish + horizontal band)
      - inside seed: PUSH S/V strongly toward target (S~100, V~90+)
      - optional stronger hue pull toward blue center
    """

    hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV).astype(np.float32)
    H = hsv[..., 0]
    S = hsv[..., 1]
    V = hsv[..., 2]

    # -----------------------------
    # (1) Robust normalize V (stronger)
    # -----------------------------
    v1 = np.percentile(V, 1)
    v2 = np.percentile(V, 99)
    if v2 > v1 + 1e-3:
        Vn = (V - v1) * (255.0 / (v2 - v1))
        Vn = np.clip(Vn, 0, 255)
    else:
        Vn = V.copy()
    
    V8 = Vn.astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit=6.0, tileGridSize=(5, 5))
    Vc = clahe.apply(V8).astype(np.float32)

    gamma = 0.3   # was 0.65, now harsher
    Vg = 255.0 * np.power(np.clip(Vc / 255.0, 0, 1), gamma)

    # slight extra lift to midtones
    Vg = np.clip(Vg + 10.0, 0, 255)

    hsv2 = hsv.copy()
    hsv2[..., 2] = Vg
    # -----------------------------
    hsv2_u8 = hsv2.astype(np.uint8)



    lower_seed = np.array([70,  0,  0], dtype=np.uint8)
    upper_seed = np.array([135, 255, 255], dtype=np.uint8)
    seed0 = cv2.inRange(hsv2_u8, lower_seed, upper_seed)


    S_min_after = 15
    seed0 = seed0 & ((hsv2_u8[...,1] >= S_min_after).astype(np.uint8) * 255)

    # cleanup
    seed0 = cv2.medianBlur(seed0, 5)

    # horizontal emphasis: open with long horizontal kernel
    seed1 = cv2.morphologyEx(
        seed0, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (15, 3)),
        iterations=1
    )

    # connect horizontally
    seed1 = cv2.morphologyEx(
        seed1, cv2.MORPH_CLOSE,
        cv2.getStructuringElement(cv2.MORPH_RECT, (25, 5)),
        iterations=1
    )

    seed01 = (seed1 > 0)

    # -----------------------------
    # (5) EXTREME push of S/V inside seed
    # target-ish: S ~ 95-100, V ~ 80-100
    # -----------------------------
    S2 = S.copy()
    V2 = Vg.copy()

    # Push S upward HARD:
    #   - If S is tiny, multiplication does nothing, so we also ADD a lot
    S2[seed01] = np.clip(S2[seed01] * 2.8 + 80, 0, 255)

    # Push V upward HARD (but still capped):
    V2[seed01] = np.clip(V2[seed01] * 1.35 + 35, 0, 255)

    # Optional: clamp seed area to minimum "good" S/V
    # (forces your desired style)
    S2[seed01] = np.maximum(S2[seed01], 255)   # ~90% of 255
    V2[seed01] = np.maximum(V2[seed01], 230)   # ~75% of 255

    # -----------------------------
    # (6) Stronger hue pull toward blue center
    # -----------------------------
    H2 = H.copy()
    blue_center = 100.0
    pull = 0.55   # was 0.25, now harsher

    good = seed01  # since we already enforced S_min_after
    H2[good] = (1 - pull) * H2[good] + pull * blue_center
    H2 = np.clip(H2, 0, 179)

    hsv_out = np.stack([H2, S2, V2], axis=-1).astype(np.uint8)
    out_rgb = cv2.cvtColor(hsv_out, cv2.COLOR_HSV2RGB)

    if debug:
        nseed = int(seed01.sum())
        print(f"[RESCUE EXTREME] seed_pixels={nseed} gamma={gamma} clahe=4.0 pull={pull}")

    return out_rgb


New_Origignal_Img1111 = io.imread("Localization Test/NewTestCases/white car harsh right angle.jpg")
New_Origigna2_Img1111 = io.imread("Localization Test/NewTestCases/white car harshest angle.jpg")
New_Origigna3_Img1111 = io.imread("Localization Test/NewTestCases/white car slight angle .jpg")
New_Origigna4_Img1111 = io.imread("Localization Test/NewTestCases/white car straight.jpg")
New_Origigna5_Img1111 = io.imread("Localization Test/NewTestCases/white car very elevated angle.jpg")
New_Origigna6_Img1111 = io.imread("Localization Test/NewTestCases/white car very harsh left angle with alot of noise.jpg")

# New_Origigna7_Img1111 = io.imread("Localization Test/NewTestCases/angle no bright.jpeg")
New_Origigna8_Img1111 = io.imread("Localization Test/NewTestCases/blury gedan.jpeg")
New_Origigna9_Img1111 = io.imread("Localization Test/NewTestCases/bright with angle.jpeg")
New_Origigna12_Img1111 = io.imread("Localization Test/NewTestCases/very dirty and noisy.jpeg")
New_Origigna13_Img1111 = io.imread("Localization Test/NewTestCases/very light bright.jpeg")
New_Origigna14_Img1111 = io.imread("Localization Test/NewTestCases/veryyyy far.jpeg")
New_Origigna14_Img111411111 = io.imread("WhatsApp Image 2025-12-18 at 1.32.03 PM.jpeg")


# cropped_Plate1=debug_blue_crop(New_Origigna14_Img111111111,"Detected_New_Origignal_Img1111") 
# cropped_Plate1=debug_blue_crop(cropped_Plate1,"Detected_New_Origignal_Img1111",False)

# cropped_Plate1=debug_blue_crop(New_Origigna14_Img111111211,"Detected_New_Origignal_Img1111") 
# cropped_Plate1=debug_blue_crop(cropped_Plate1,"Detected_New_Origignal_Img1111",False)

# cropped_Plate1=debug_blue_crop(New_Origigna14_Img111411111,"Detected_New_Origignal_Img1111") 
# cropped_Plate1=debug_blue_crop(cropped_Plate1,"Detected_New_Origignal_Img1111",False)

# cropped_Plate1=debug_blue_crop(New_Origignal_Img1111,"Detected_New_Origignal_Img1111")
# cropped_Plate1=debug_blue_crop(cropped_Plate1,"Detected_New_Origignal_Img1111",False) 

# cropped_Plate2=debug_blue_crop(New_Origigna2_Img1111,"Detected_New_Origigna2_Img1111")    
# cropped_Plate2=debug_blue_crop(cropped_Plate2,"Detected_New_Origigna2_Img1111",False)  

# cropped_Plate3=debug_blue_crop(New_Origigna3_Img1111,"Detected_New_Origigna3_Img1111")    
# cropped_Plate3=debug_blue_crop(cropped_Plate3,"Detected_New_Origigna3_Img1111",False)    

# cropped_Plate4=debug_blue_crop(New_Origigna4_Img1111,"Detected_New_Origigna4_Img1111")    
# cropped_Plate4=debug_blue_crop(cropped_Plate4,"Detected_New_Origigna4_Img1111",False) 
# cropped_Plate5=debug_blue_crop(New_Origigna5_Img1111,"Detected_New_Origigna5_Img1111")    
# cropped_Plate5=debug_blue_crop(cropped_Plate5,"Detected_New_Origigna5_Img1111",False)     
# cropped_Plate6=debug_blue_crop(New_Origigna6_Img1111,"Detected_New_Origigna6_Img1111")
# cropped_Plate6=debug_blue_crop(cropped_Plate6,"Detected_New_Origigna6_Img1111",False)  

# cropped_Plate7=debug_blue_crop(Origignal_Img1,"Detected_New_Origignal_Img1111")    
# cropped_Plate7=debug_blue_crop(cropped_Plate7,"Detected_New_Origignal_Img1111",False)    
# cropped_Plate8=debug_blue_crop(Origignal_Img2,"Detected_New_Origigna2_Img1111")    
# cropped_Plate8=debug_blue_crop(cropped_Plate8,"Detected_New_Origigna2_Img1111",False)    
# cropped_Plate9=debug_blue_crop(Origignal_Img3,"Detected_New_Origigna3_Img1111")    
# cropped_Plate9=debug_blue_crop(cropped_Plate9,"Detected_New_Origigna3_Img1111",False)     
# cropped_Plate10=debug_blue_crop(Origignal_Img4,"Detected_New_Origigna4_Img1111")    
# cropped_Plate10=debug_blue_crop(cropped_Plate10,"Detected_New_Origigna4_Img1111",False)      
# cropped_Plate11=debug_blue_crop(Origignal_Img5,"Detected_New_Origigna5_Img1111")
# cropped_Plate11=debug_blue_crop(cropped_Plate11,"Detected_New_Origigna5_Img1111",False)


# cropped_Plate12=debug_blue_crop(Origignal_Img11,"Detected_New_Origigna6_Img1111") 
# cropped_Plate12=debug_blue_crop(cropped_Plate12,"Detected_New_Origigna6_Img1111",False)  
# cropped_Plate13=debug_blue_crop(Origignal_Img12,"Detected_New_Origignal_Img1111")    
# cropped_Plate13=debug_blue_crop(cropped_Plate13,"Detected_New_Origignal_Img1111",False)    
# cropped_Plate14=debug_blue_crop(Origignal_Img13,"Detected_New_Origigna2_Img1111")    
# cropped_Plate14=debug_blue_crop(cropped_Plate14,"Detected_New_Origigna2_Img1111",False)      
# cropped_Plate15=debug_blue_crop(Origignal_Img14,"Detected_New_Origigna3_Img1111")    
# cropped_Plate15=debug_blue_crop(cropped_Plate15,"Detected_New_Origigna3_Img1111",False)    
# cropped_Plate16=debug_blue_crop(Origignal_Img15,"Detected_New_Origigna4_Img1111")
# cropped_Plate16=debug_blue_crop(cropped_Plate16,"Detected_New_Origigna4_Img1111",False)

# cropped_Plate17=debug_blue_crop(Origignal_Img111,"Detected_New_Origigna5_Img1111")    
# cropped_Plate17=debug_blue_crop(cropped_Plate17,"Detected_New_Origigna5_Img1111",False)     
# cropped_Plate18=debug_blue_crop(Origignal_Img112,"Detected_New_Origigna6_Img1111") 
# cropped_Plate18=debug_blue_crop(cropped_Plate18,"Detected_New_Origigna6_Img1111",False) 
# cropped_Plate19=debug_blue_crop(Origignal_Img113,"Detected_New_Origignal_Img1111")    
# cropped_Plate19=debug_blue_crop(cropped_Plate19,"Detected_New_Origignal_Img1111",False)    

# cropped_Plate22=debug_blue_crop(Origignal_Img1111,"Detected_New_Origigna4_Img1111")    
# cropped_Plate22=debug_blue_crop(cropped_Plate22,"Detected_New_Origigna4_Img1111",False)   
# cropped_Plate23=debug_blue_crop(Origignal_Img1112,"Detected_New_Origigna5_Img1111")    
# cropped_Plate23=debug_blue_crop(cropped_Plate23,"Detected_New_Origigna5_Img1111",False)    
# cropped_Plate24=debug_blue_crop(Origignal_Img1113,"Detected_New_Origigna6_Img1111") 
# cropped_Plate24=debug_blue_crop(cropped_Plate24,"Detected_New_Origigna6_Img1111",False)   
# cropped_Plate25=debug_blue_crop(Origignal_Img1114,"Detected_New_Origignal_Img1111")    
# cropped_Plate25=debug_blue_crop(cropped_Plate25,"Detected_New_Origignal_Img1111",False)      
 

# cropped_Plate27=debug_blue_crop(Original_Img11111,"Detected_New_Origigna4_Img1111")    
# cropped_Plate27=debug_blue_crop(cropped_Plate27,"Detected_New_Origigna4_Img1111",False)    
# cropped_Plate28=debug_blue_crop(Original_Img11112,"Detected_New_Origigna5_Img1111")    
# cropped_Plate28=debug_blue_crop(cropped_Plate28,"Detected_New_Origigna5_Img1111",False)     
# cropped_Plate29=debug_blue_crop(Original_Img11113,"Detected_New_Origigna6_Img1111") 
# cropped_Plate29=debug_blue_crop(cropped_Plate29,"Detected_New_Origigna6_Img1111",False)  
# cropped_Plate30=debug_blue_crop(Original_Img11114,"Detected_New_Origignal_Img1111")    
# cropped_Plate30=debug_blue_crop(cropped_Plate30,"Detected_New_Origignal_Img1111",False)    
# cropped_Plate31=debug_blue_crop(Original_Img11115,"Detected_New_Origigna2_Img1111") 
# cropped_Plate31=debug_blue_crop(cropped_Plate31,"Detected_New_Origigna2_Img1111",False)   

# # cropped_Plate32=debug_blue_crop(New_Origigna7_Img1111,"Detected_New_Origigna4_Img1111")    
# # cropped_Plate32=debug_blue_crop(cropped_Plate32,"Detected_New_Origigna4_Img1111",False)   
# cropped_Plate33=debug_blue_crop(New_Origigna8_Img1111,"Detected_New_Origigna5_Img1111")    
# cropped_Plate33=debug_blue_crop(cropped_Plate33,"Detected_New_Origigna5_Img1111",False)      
# cropped_Plate34=debug_blue_crop(New_Origigna9_Img1111,"Detected_New_Origigna6_Img1111") 
# cropped_Plate34=debug_blue_crop(cropped_Plate34,"Detected_New_Origigna6_Img1111",False)   
# cropped_Plate37=debug_blue_crop(New_Origigna12_Img1111,"Detected_New_Origigna4_Img1111")    
# cropped_Plate37=debug_blue_crop(cropped_Plate37,"Detected_New_Origigna4_Img1111",False)    
# cropped_Plate38=debug_blue_crop(New_Origigna13_Img1111,"Detected_New_Origigna5_Img1111")    
# cropped_Plate38=debug_blue_crop(cropped_Plate38,"Detected_New_Origigna5_Img1111",False)    
# cropped_Plate39=debug_blue_crop(New_Origigna14_Img1111,"Detected_New_Origigna6_Img1111") 
# cropped_Plate39=debug_blue_crop(cropped_Plate39,"Detected_New_Origigna6_Img1111",False)










# import os
# from skimage import io

# BASE_PATH = "Localization Test/Vehicles"

# START_ID = 1
# END_ID   = 2087

# for i in range(START_ID, END_ID,1):
#     import matplotlib.pyplot as plt
#     plt.ioff()  # disable interactive rendering
#     fname = f"{i:04d}.jpg"        # 0001.jpg, 0002.jpg, ...
#     fpath = os.path.join(BASE_PATH, fname)

#     if not os.path.exists(fpath):
#         print(f"[SKIP] Missing: {fpath}")
#         continue

#     try:
#         img = io.imread(fpath)    # RGB (as you want)
#         title = f"Vehicle_{fname}"
#         print(f"[RUN] {title}")
#         img1=debug_blue_crop(img, title)
#         img2=debug_blue_crop(img1, title,False)

#     except Exception as e:
#         print(f"[ERROR] {fname}: {e}")




import os
from skimage import io
import matplotlib.pyplot as plt

BASE_PATH = "Localization Test/Vehicles"
OUT_DIR   = "Localization Test/FinalCrops"
os.makedirs(OUT_DIR, exist_ok=True)

IDS = [
    2087, 2084, 2083, 2082, 2081, 2080, 2079, 2078,
    2076, 2073, 2072, 2070, 2069, 2068, 2066, 2065,
    2063, 2062, 2060, 2058, 2057, 2056, 2055, 2054,
    2051, 2049, 2047, 2044, 2043, 2041,
    2039, 2038, 2037, 2036, 2035, 2034,
    2033, 2031, 2030, 2029, 2026, 2025,

    1, 2, 6, 10, 13, 14, 15, 16, 25, 33, 34, 35,
    39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
    51, 52, 57, 58, 59, 60, 61, 62, 63, 64, 65,
    124
]

plt.ioff()

for i in IDS:
    fname = f"{i:04d}.jpg"
    fpath = os.path.join(BASE_PATH, fname)

    if not os.path.exists(fpath):
        print(f"[SKIP] Missing: {fname}")
        continue

    try:
        img = io.imread(fpath)
        title = f"Vehicle_{fname}"
        print(f"[RUN] {title}")

        img1 = debug_blue_crop(img, title, enable_perspective=True)
        img2 = debug_blue_crop(img1, title, enable_perspective=False)

        if img2 is not None:
            out_path = os.path.join(OUT_DIR, f"{i:04d}_final.png")
            io.imsave(out_path, img2)
            print(f"[SAVE] {out_path}")

    except Exception as e:
        print(f"[ERROR] {fname}: {e}")





In [ ]:
'''
import pytesseract
import cv2
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import os
import easyocr
os.environ['TESSDATA_PREFIX'] = os.path.abspath("Dataset")
# os.environ.pop('TESSDATA_PREFIX', None)
def add_padding(img, pad=10):
    return cv2.copyMakeBorder(
        img, pad, pad, pad, pad,
        borderType=cv2.BORDER_CONSTANT,
        value=255
    )

def add_paddingblack(img, pad=200):
    return cv2.copyMakeBorder(
        img, pad, pad, pad, pad,
        borderType=cv2.BORDER_CONSTANT,
        value=0
    )

def recognize_arabic_digit(image_path, show_preprocessed=False):
    """
    Recognize a single Arabic digit from an image.
    
    Args:
        image_path (str): Path to image file.
        show_preprocessed (bool): If True, shows the binary preprocessed image.
    
    Returns:
        str: Detected Arabic digit or empty string if not recognized.
    """
    
    # 1. Load image
    # img = io.imread(image_path)
    # img=add_paddingblack(img)
    # img=add_padding(img)
    img=d_mask
    
    # # 2. Convert to grayscale if needed
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
    
    # # 3. Resize image up to help OCR
    # gray = cv2.resize(gray, None, fx=5, fy=5, interpolation=cv2.INTER_LINEAR)
    
    # # 4. Smooth image to reduce noise
    # blur = cv2.GaussianBlur(gray, (3,3), 0)
    
    # # 5. Threshold to get binary image
    # _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # # 6. Invert if background is dark
    # if np.mean(thresh) < 127:
    #     thresh = cv2.bitwise_not(thresh)
    
    # Optional: show preprocessed image
    if show_preprocessed:
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.show()
    
    # 7. Configure Tesseract for single character + Arabic digits only
    # config = f'--oem 3 --psm 10 -c tessedit_char_whitelist={ARABIC_DIGITS}'
    
    # 8. Run OCR
    # text = pytesseract.image_to_string(thresh, lang='ara')

    reader = easyocr.Reader(['ar'])
    results = reader.readtext(img)
    texts = [text for bbox, text, prob in results]


    # 9. Clean result
    # text = text.strip()
    return texts

# Example usage:
digit = recognize_arabic_digit("letter_0.png", show_preprocessed=True)
print("Detected Arabic digit EASYOCR:", repr(digit))
# print("Detected Arabic digit:", repr(digit))

'''

In [ ]:
'''
MAAW NEW SEG
'''


import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
import os
import glob


def extract_plate(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 80, 200)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    cnts, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(cnts, key=cv2.contourArea)

    x,y,w,h = cv2.boundingRect(cnt)
    return img[y:y+h, x:x+w]


def crop_digits_letters_region(plate):
    h, w = plate.shape[:2]

    # Egyptian plate layout: bottom ~65% contains digits + letters
    y0 = int(0.35 * h)
    y1 = int(0.95 * h)

    return plate[y0:y1, :]




def split_digits_letters(region_bgr):
    h, w = region_bgr.shape[:2]

    # Egyptian plate layout rule (robust to tilt)
    split_col = int(0.58 * w)

    digits = region_bgr[:, :split_col]
    letters = region_bgr[:, split_col:]

    return digits, letters




'''
def get_components(region):
    gray = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)

    # Improve contrast
    gray = cv2.GaussianBlur(gray, (5,5), 0)

    # Global threshold (much cleaner)
    _, bin_img = cv2.threshold(
        gray, 0, 255,
        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    # Remove small noise
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, kernel)

    # Merge letter dots
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_CLOSE, kernel)

    labeled = label(bin_img)
    boxes = []

    H, W = bin_img.shape
    for r in regionprops(labeled):
        if r.area < 500:
            continue
        y0, x0, y1, x1 = r.bbox
        h, w = y1 - y0, x1 - x0
        if h < 0.35*H or w < 0.05*W:
            continue
        boxes.append((x0,y0,x1,y1))

    boxes = sorted(boxes, key=lambda b: b[0])

    # Enforce Egyptian plate rules
    if len(boxes) > 3:
        # Keep the 3 widest components (letters are wide)
        boxes = sorted(boxes, key=lambda b: (b[2] - b[0]), reverse=True)[:3]
        boxes = sorted(boxes, key=lambda b: b[0])


    if len(boxes) > 4:
        boxes = sorted(boxes, key=lambda b: (b[2] - b[0]), reverse=True)[:4]
        boxes = sorted(boxes, key=lambda b: b[0])




    return bin_img, boxes
'''

def touches_border(x0, y0, x1, y1, W, H, margin=2):
    return (x0 <= margin) or (y0 <= margin) or (x1 >= W - margin) or (y1 >= H - margin)

def get_components(region_bgr, kind="digits"):
    gray = cv2.cvtColor(region_bgr, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)

    # Clean binary mask (white chars on black)
    _, bin_img = cv2.threshold(
        gray, 0, 255,
        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    # Small noise removal (keep strokes)
    bin_img = cv2.morphologyEx(
        bin_img,
        cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    )

    # DIFFERENT closing for digits vs letters
    if kind == "letters":
        # Connect Arabic dots + separated parts vertically (important for ى, ي, ن, ق...)
        close_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 11))  # tall vertical
    else:
        # Digits: avoid merging neighbors
        close_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 7))

    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_CLOSE, close_kernel)

    labeled = label(bin_img)
    boxes = []

    H, W = bin_img.shape

    # Area thresholds (digits can be smaller than letters)
    min_area = 180 if kind == "digits" else 300

    for r in regionprops(labeled):
        if r.area < min_area:
            continue

        y0, x0, y1, x1 = r.bbox
        h = y1 - y0
        w = x1 - x0

        # Reject border fragments (fixes the right frame being a "letter")
        if touches_border(x0, y0, x1, y1, W, H, margin=2):
            continue

        # Basic size constraints
        if h < 0.35 * H:
            continue
        if w < 0.03 * W:
            continue

        # Reject thin vertical frame-like components
        aspect = w / float(h)
        if aspect < 0.08:   # very thin & tall => border / separator
            continue

        boxes.append((x0, y0, x1, y1))

    # left->right
    boxes = sorted(boxes, key=lambda b: b[0])
    return bin_img, boxes


def draw_boxes(mask, boxes):
    out = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    for x0,y0,x1,y1 in boxes:
        cv2.rectangle(out,(x0,y0),(x1,y1),(255,0,0),2)
    return out


def extract_chars(region, boxes, reverse=False):
    boxes = sorted(boxes, key=lambda b: b[0], reverse=reverse)
    chars = []
    for x0,y0,x1,y1 in boxes:
        chars.append(region[y0:y1, x0:x1])
    return chars



def trim_borders(img, px=6):
    h, w = img.shape[:2]
    return img[px:h-px, px:w-px]





img_bgr = cropped_Plate7














img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

plate = extract_plate(img_bgr)

dl_region = crop_digits_letters_region(plate)
dl_region = trim_borders(dl_region, px=4)

# FINAL robust split
digits_region, letters_region = split_digits_letters(dl_region)

d_mask, d_boxes = get_components(digits_region, kind="digits")
l_mask, l_boxes = get_components(letters_region, kind="letters")


# Keep at most 4 digits: choose by area (largest blobs)
if len(d_boxes) > 4:
    d_boxes = sorted(d_boxes, key=lambda b: (b[2]-b[0])*(b[3]-b[1]), reverse=True)[:4]
    d_boxes = sorted(d_boxes, key=lambda b: b[0])

# Keep at most 3 letters: choose by area (largest blobs)
if len(l_boxes) > 3:
    l_boxes = sorted(l_boxes, key=lambda b: (b[2]-b[0])*(b[3]-b[1]), reverse=True)[:3]
    l_boxes = sorted(l_boxes, key=lambda b: b[0])





BASE_OUT_DIR = "output_chars"
#BASE_OUT_DIR = "Image-Processing-Project"
DIGITS_DIR = os.path.join(BASE_OUT_DIR, "digits")
LETTERS_DIR = os.path.join(BASE_OUT_DIR, "letters")

os.makedirs(DIGITS_DIR, exist_ok=True)
os.makedirs(LETTERS_DIR, exist_ok=True)

def clear_old_files(folder, pattern):
    for f in glob.glob(os.path.join(folder, pattern)):
        os.remove(f)

# Clear previous results
clear_old_files(DIGITS_DIR, "digit_*.png")
clear_old_files(LETTERS_DIR, "letter_*.png")





digits = extract_chars(d_mask, d_boxes, reverse=False)
letters = extract_chars(l_mask, l_boxes, reverse=True)
TARGET_SIZE = (150, 150)

# Save digits
for i, d in enumerate(digits):
    d_resized = cv2.resize(d, TARGET_SIZE, interpolation=cv2.INTER_AREA)
    save_path = os.path.join(DIGITS_DIR, f"digit_{i}.png")
    cv2.imwrite(save_path, d_resized)

# Save letters
for i, l in enumerate(letters):
    l_resized = cv2.resize(l, TARGET_SIZE, interpolation=cv2.INTER_AREA)
    save_path = os.path.join(LETTERS_DIR, f"letter_{i}.png")
    cv2.imwrite(save_path, l_resized)





d_boxes_img = draw_boxes(d_mask, d_boxes)
l_boxes_img = draw_boxes(l_mask, l_boxes)


show_images(
    [
        img_rgb,
        cv2.cvtColor(plate, cv2.COLOR_BGR2RGB),
        cv2.cvtColor(dl_region, cv2.COLOR_BGR2RGB),
        cv2.cvtColor(digits_region, cv2.COLOR_BGR2RGB),
        cv2.cvtColor(letters_region, cv2.COLOR_BGR2RGB)
    ],
    ["Original", "Plate Crop", "Digits+Letters", "Digits Region", "Letters Region"]
)


show_images([d_mask, l_mask], ["Digits Mask", "Letters Mask"])
show_images([d_boxes_img, l_boxes_img], ["Digits Boxes", "Letters Boxes"])

show_images(digits, [f"Digit {i+1}" for i in range(len(digits))])
show_images(letters, [f"Letter {i+1}" for i in range(len(letters))])










'''
MAAW NEW SEG
'''

In [ ]:
'''
MAAW SEGMENTATION
'''
'''




import numpy as np
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage.color import rgb2gray, rgba2rgb
from skimage.filters import threshold_otsu
from skimage.morphology import (
    remove_small_holes,
    remove_small_objects,
    binary_closing,
    square,
)
from skimage.measure import label, regionprops
from skimage.transform import resize
from skimage.color import gray2rgb


def crop_bottom(img, top_ratio=0.40):
    h, w = img.shape[:2]
    return img[int(h * top_ratio):, :]


def split_digits_letters(bottom):
    h, w = bottom.shape[:2]
    mid = w // 2
    return bottom[:, :mid], bottom[:, mid:]


def get_mask(region):
    gray = rgb2gray(region)

    T = threshold_otsu(gray)
    mask = gray < T

    mask = remove_small_objects(mask, 25)

    return mask






def touches_border(r0, c0, r1, c1, H, W, margin_ratio=0.005):
    mr = int(margin_ratio * H)
    mc = int(margin_ratio * W)

    if r0 <= mr: return True        # top
    if c0 <= mc: return True        # left
    if r1 >= H - mr: return True    # bottom
    if c1 >= W - mc: return True    # right

    return False



def extract_digits(digits_region, max_chars=4, debug=False):
    H0, W0 = digits_region.shape[:2]
    border_ratio = 0.04
    by = int(border_ratio * H0)
    bx = int(border_ratio * W0)
    sub = digits_region[by:H0 - by, bx:W0 - bx]
    
    mask = get_mask(sub)
    H, W = mask.shape
    
    def remove_frame_components(mask):
        H, W = mask.shape
        lbl = label(mask)

        for p in regionprops(lbl):
            r0, c0, r1, c1 = p.bbox
            h = r1 - r0
            w = c1 - c0

            wr = w / (W + 1e-6)
            hr = h / (H + 1e-6)

            near_edge = (r0 < 0.20 * H) or (r1 > 0.80 * H) #very wide, thin, near top or bottom

            if near_edge and wr > 0.75 and hr < 0.12:
                mask[lbl == p.label] = 0

        return mask

    mask = remove_frame_components(mask)
    
    def touches_top_or_bottom(r0, r1, H, margin=2):
        return r0 <= margin or r1 >= H - margin

    def is_horizontal_frame(h, w, H, W, r0, r1):
        very_wide = w > 0.65 * W
        very_thin = h < 0.12 * H
        near_edge = r0 < 0.15 * H or r1 > 0.85 * H
        return very_wide and very_thin and near_edge



    lbl = label(mask)

    candidates = []
    for p in regionprops(lbl):
        r0, c0, r1, c1 = p.bbox

        if touches_top_or_bottom(r0, r1, H):
            continue

        h = r1 - r0
        w = c1 - c0
        area = p.area
        wr = w / W
        ar = area / (H * W)

        if is_horizontal_frame(h, w, H, W, r0, r1):
            continue

        if wr < 0.035 or wr > 0.75:
            continue

        if ar < 0.001:
            continue

        candidates.append({"bbox": [r0, c0, r1, c1], "area": area})

    candidates.sort(key=lambda c: c["bbox"][1])


    merged = []
    for c in candidates:
        if not merged:
            merged.append(c)
            continue

        r0, c0, r1, c1 = c["bbox"]
        r0m, c0m, r1m, c1m = merged[-1]["bbox"]

        overlap_x = min(c1, c1m) - max(c0, c0m)
        overlap_y = min(r1, r1m) - max(r0, r0m)

        #only merge if they overlap in X and Y or very close vertically
        vertical_gap = max(r0, r0m) - min(r1, r1m) #negative means it overlaps

        if overlap_x > 0 and (overlap_y > 0 or vertical_gap < 0.08 * H):
            merged[-1]["bbox"] = [
                min(r0, r0m), min(c0, c0m),
                max(r1, r1m), max(c1, c1m)
            ]
        else:
            merged.append(c)

    boxed_mask = gray2rgb(mask.astype(np.uint8) * 255)

    chars = []
    for m in merged:
        r0, c0, r1, c1 = m["bbox"]
        if (r1 - r0) < 0.22 * H:
            continue

        boxed_mask[r0:r1, [c0, c1-1]] = [255, 0, 0]
        boxed_mask[[r0, r1-1], c0:c1] = [255, 0, 0]

        crop = resize(mask[r0:r1, c0:c1].astype(float), (40, 40))
        chars.append((c0 + bx, crop))

    chars.sort(key=lambda x: x[0])

    if debug:
        return [c for _, c in chars][:max_chars], mask, boxed_mask
    else:
        return [c for _, c in chars][:max_chars]



def extract_letters(letters_region, max_chars=3, debug=False):
    H0, W0 = letters_region.shape[:2]
    by = int(0.04 * H0)
    bx = int(0.04 * W0)
    sub = letters_region[by:H0 - by, bx:W0 - bx]

    mask = get_mask(sub)
    H, W = mask.shape
    lbl = label(mask)

    boxed_mask = gray2rgb(mask.astype(np.uint8) * 255)

    big_blobs = []   #letter body
    small_blobs = [] #dots


    for p in regionprops(lbl):
        r0, c0, r1, c1 = p.bbox

        if touches_border(r0, c0, r1, c1, H, W, margin_ratio=0.02):
            continue
    
        h = r1 - r0
        w = c1 - c0
        area = p.area
        cy = p.centroid[0] / H
        wr = w / W
        ar = area / (H * W)

        #dot (area < 0.015)
        if ar < 0.015:
            small_blobs.append((r0, c0, r1, c1))
            continue

        #main letter filtering
        if wr < 0.08 or wr > 0.75:
            continue
        if ar > 0.22:
            continue
        if cy > 0.75:
            continue

        big_blobs.append([r0, c0, r1, c1])


    for dr0, dc0, dr1, dc1 in small_blobs:
        dc = (dc0 + dc1) / 2
        dr = (dr0 + dr1) / 2

        best_i = -1
        best_dx = 1e9

        for i, box in enumerate(big_blobs):
            r0, c0, r1, c1 = box
            bc = (c0 + c1) / 2
            br = (r0 + r1) / 2

            dx = abs(dc - bc)
            dy = abs(dr - br)

            #vertically and horizontally closest
            if dx < best_dx and dy < 120:
                best_dx = dx
                best_i = i

        #attach dot to nearest valid letter ONLY
        if best_i != -1:
            box = big_blobs[best_i]
            box[0] = min(box[0], dr0)
            box[1] = min(box[1], dc0)
            box[2] = max(box[2], dr1)
            box[3] = max(box[3], dc1)


    big_blobs.sort(key=lambda b: b[1])


    #new
    # ---------- MERGE BROKEN LETTER BODIES (e.g. س) ----------
    merged = []
    for box in sorted(big_blobs, key=lambda b: b[1]):
        if not merged:
            merged.append(box)
            continue

        r0, c0, r1, c1 = box
        pr0, pc0, pr1, pc1 = merged[-1]

        # horizontal gap
        gap = c0 - pc1

        # vertical overlap
        overlap = min(r1, pr1) - max(r0, pr0)

        if gap < 0.15 * W and overlap > 0.3 * min(r1 - r0, pr1 - pr0):
            # merge
            merged[-1] = [
                min(r0, pr0),
                min(c0, pc0),
                max(r1, pr1),
                max(c1, pc1)
            ]
        else:
            merged.append(box)

    big_blobs = merged

    #new





    chars = []
    for r0, c0, r1, c1 in big_blobs:
        boxed_mask[r0:r1, [c0, c1-1]] = [255, 0, 0]
        boxed_mask[[r0, r1-1], c0:c1] = [255, 0, 0]

        crop = resize(mask[r0:r1, c0:c1].astype(float), (40, 40))
        chars.append((c0 + bx, crop))

    if debug:
        return [c for _, c in chars][:max_chars], mask, boxed_mask
    else:
        return [c for _, c in chars][:max_chars]



def process_plate(img):
    bottom = crop_bottom(img)
    digits_region, letters_region = split_digits_letters(bottom)

    digits, d_mask, d_boxes = extract_digits(digits_region, debug=True)
    letters, l_mask, l_boxes = extract_letters(letters_region, debug=True)

    return digits_region, letters_region, digits, letters, d_mask, d_boxes, l_mask, l_boxes


img22 = imread('Q1.jpg')
#img22 = imread('Dataset/Buses & government vehicles.png')
#img22 = imread('Dataset/Diplomatic vehicles.png')
#img22 = imread('Dataset/image.png')
#img22 = imread('Dataset/Limousines & tourist buses.png')
#img22 = imread('Dataset/Police vehicles.png')
# img22 = rgba2rgb(imread('Dataset/Private vehicles & motorcycles.png'))
#img22 = rgba2rgb(imread('Dataset/Taxis.png'))
#img22 = imread('Dataset/Trucks.png')
#img22 = imread('Dataset/Vehicles with unpaid customs.png')

#etsh cropped tests########################
# img22 = cropped_Plate1
# img22 = cropped_Plate2 #blur problem
#img22 = rgba2rgb(imread('Q1C.png'))
# img22 = cropped_Plate3 #blur problem
# img22 = cropped_Plate5 #FIXED frame counting as letter from left
# img22 = cropped_Plate11 #FIXED digit border on top detected  
# img22 = cropped_Plate12 #FIXED border in digit detected
# img22 = cropped_Plate13 #FIXED border detected as letter
#img22 = cropped_Plate15 #blur problem
#img22 = cropped_Plate111 ##
#img22 = cropped_Plate112 #blur problem
#img22 = cropped_Plate113 #blur problem
#img22 = cropped_Plate115 ##
#img22 = cropped_Plate1111 #hard angle
#img22 = cropped_Plate1112 #hard angle
#img22 = cropped_Plate1113 #blur problem
#img22 = cropped_Plate1115 #blur
#img22 = cropped_Plate11113 #blur
#img22 = cropped_Plate11114 ##
#img22 = cropped_Plate11115 ##
#img22 = cropped_Plate11116 ##
#img22 = cropped_Plate11117 ##

#etsh cropped tests########################




import os

def clear_previous_outputs():
    files = [
        "digit_0.png", "digit_1.png", "digit_2.png", "digit_3.png", "digit_4.png", "letter_0.png", "letter_1.png", "letter_2.png"
    ]

    for f in files:
        if os.path.exists(f):
            os.remove(f)


def save_characters(digits, letters):
    #digits
    for i, img in enumerate(digits):
        out = (img * 255).astype(np.uint8) if img.dtype != np.uint8 else img
        cv2.imwrite(f"digit_{i}.png", out)

    #letters
    for i, img in enumerate(letters):
        out = (img * 255).astype(np.uint8) if img.dtype != np.uint8 else img
        cv2.imwrite(f"letter_{i}.png", out)


clear_previous_outputs()

d_reg, l_reg, digits, letters, d_mask, d_boxes, l_mask, l_boxes = process_plate(img22)






show_images([d_reg, l_reg], ["Digits Region", "Letters Region"])

show_images([d_mask, l_mask], ["Digits Mask", "Letters Mask"])

show_images([d_boxes, l_boxes], ["Digits Mask + Boxes", "Letters Mask + Boxes"])

digit_titles = [f"Digit {i}" for i in range(len(digits))]
show_images(digits, digit_titles)

letter_titles = [f"Letter {i}" for i in range(len(letters))]
show_images(letters, letter_titles)


save_characters(digits,letters)


#RUN LOOP

for i in range(1, 40):
    img = globals()[f"cropped_Plate{i}"]

    (
        d_reg, l_reg,
        digits, letters,
        d_mask, d_boxes,
        l_mask, l_boxes
    ) = process_plate(img)

    show_images([d_reg, l_reg], ["Digits Region", "Letters Region"])
    show_images([d_mask, l_mask], ["Digits Mask", "Letters Mask"])
    show_images([d_boxes, l_boxes], ["Digits Mask + Boxes", "Letters Mask + Boxes"])

    digit_titles = [f"Digit {j}" for j in range(len(digits))]
    show_images(digits, digit_titles)

    letter_titles = [f"Letter {j}" for j in range(len(letters))]
    show_images(letters, letter_titles)


#RUN LOOP


'''
'''
MAAW SEGMENTATION
'''
        

In [ ]:
'''
MAAW DIGIT AND LETTER RECOGNITION
'''
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import os


english_to_arabic = {
    1: '١',
    2: '٢',
    3: '٣',
    4: '٤',
    5: '٥',
    6: '٦',
    7: '٧',
    8: '٨',
    9: '٩'
    }


def normalize_digit(img):
    img = img.copy()

    if np.mean(img) > 127:
        img = cv2.bitwise_not(img)

    ys, xs = np.where(img > 0)
    if len(xs) == 0:
        return cv2.resize(img, (64, 64))

    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    img = img[y0:y1+1, x0:x1+1]

    img = cv2.copyMakeBorder(img, 12, 12, 12, 12, cv2.BORDER_CONSTANT, value=0)

    img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_NEAREST)
    return img.astype(np.uint8)


def match_digit(img, refs_folder="digits_reference"):
    img = normalize_digit(img)

    scores = {}

    for d in range(1,10):
        digit_folder = os.path.join(refs_folder, str(d))
        if not os.path.exists(digit_folder):
            continue

        best_local = -1

        for fname in os.listdir(digit_folder):
            ref = cv2.imread(os.path.join(digit_folder, fname), 0)
            ref = normalize_digit(ref)

            score = ssim(img, ref)
            best_local = max(best_local, score)

        scores[d] = best_local

    best_digit = max(scores, key=scores.get)

    return best_digit

final_digits_english = []
final_digits_arabic  = []

for d in digits:
    d_img = d.astype(np.uint8) 
    digit = match_digit(d_img)

    final_digits_english.append(str(digit))
    final_digits_arabic.append(english_to_arabic[digit])

print("Plate Digits In English:", "".join(final_digits_english))
print("Plate Digits In Arabic: ", "".join(final_digits_arabic))



letter_id_to_arabic = {
    1: 'أ',
    2: 'ب',
    3: 'ج',
    4: 'د',
    5: 'ر',
    6: 'ز',
    7: 'س',
    8: 'ص',
    9: 'ض',
    10: 'ط',
    11: 'ع',
    12: 'ف',
    13: 'ق',
    14: 'ك',
    15: 'ل',
    16: 'م',
    17: 'ن',
    18: 'ه',
    19: 'و',
    20: 'ي',
    21: 'ى'
}


def normalize_letter(img):
    img = img.copy()

    if np.mean(img) > 127:
        img = cv2.bitwise_not(img)

    ys, xs = np.where(img > 0)
    if len(xs) == 0:
        return cv2.resize(img, (64, 64))

    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    img = img[y0:y1+1, x0:x1+1]

    img = cv2.copyMakeBorder(
        img, 12, 12, 12, 12,
        cv2.BORDER_CONSTANT, value=0
    )

    img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_NEAREST)
    return img.astype(np.uint8)



def match_letter(img, refs_folder="letters_reference"):
    img = normalize_letter(img)

    scores = {}

    for letter_id in range(1, 22):
        letter_folder = os.path.join(refs_folder, str(letter_id))
        if not os.path.exists(letter_folder):
            continue

        best_local = -1

        for fname in os.listdir(letter_folder):
            ref = cv2.imread(os.path.join(letter_folder, fname), 0)
            if ref is None:
                continue

            ref = normalize_letter(ref)
            score = ssim(img, ref)
            best_local = max(best_local, score)

        scores[letter_id] = best_local

    best_letter_id = max(scores, key=scores.get)
    return best_letter_id


final_letters_arabic = []

for l in letters:
    l_img = l.astype(np.uint8)   
    letter_id = match_letter(l_img)


    final_letters_arabic.append(letter_id_to_arabic[letter_id])

final_plate_letters = " ".join(final_letters_arabic[::-1])

print("Plate Letters: ", final_plate_letters)



'''
MAAW DIGIT AND LETTER RECOGNITION
'''

In [ ]:
import pytesseract
import cv2
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import os
import easyocr
os.environ['TESSDATA_PREFIX'] = os.path.abspath("Dataset")
# os.environ.pop('TESSDATA_PREFIX', None)
def add_padding(img, pad=10):
    return cv2.copyMakeBorder(
        img, pad, pad, pad, pad,
        borderType=cv2.BORDER_CONSTANT,
        value=255
    )

def add_paddingblack(img, pad=50):
    return cv2.copyMakeBorder(
        img, pad, pad, pad, pad,
        borderType=cv2.BORDER_CONSTANT,
        value=0
    )


def pad_for_easyocr(img):
    h, w = img.shape
    new_img = np.zeros((h, w*3), dtype=np.uint8)  # black background
    new_img[:, w:w*2] = img  # center the character
    return new_img


def recognize_arabic_digit(image_path, show_preprocessed=False):
    """
    Recognize a single Arabic digit from an image.
    
    Args:
        image_path (str): Path to image file.
        show_preprocessed (bool): If True, shows the binary preprocessed image.
    
    Returns:
        str: Detected Arabic digit or empty string if not recognized.
    """
    
    # 1. Load image
    img = io.imread(image_path)
    # img=add_paddingblack(img)
    # img=pad_for_easyocr(img)
    
    # img=add_padding(img)
    # img=d_mask
    if img.dtype == bool:
        img = (img * 255).astype(np.uint8)

    # # Ensure grayscale, not weird format
    # if len(img.shape) == 2:
    #     img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    # # 2. Convert to grayscale if needed
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
    
    # # 3. Resize image up to help OCR
    # gray = cv2.resize(gray, None, fx=5, fy=5, interpolation=cv2.INTER_LINEAR)
    
    # # 4. Smooth image to reduce noise
    # blur = cv2.GaussianBlur(gray, (3,3), 0)
    
    # # 5. Threshold to get binary image
    # _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # # 6. Invert if background is dark
    # if np.mean(thresh) < 127:
    #     thresh = cv2.bitwise_not(thresh)
    
    # # Optional: show preprocessed image
    if show_preprocessed:
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.show()
    
    # 7. Configure Tesseract for single character + Arabic digits only
    # config = f'--oem 3 --psm 10 -c tessedit_char_whitelist={ARABIC_DIGITS}'
    
    # 8. Run OCR
    # text = pytesseract.image_to_string(thresh, lang='ara')

    reader = easyocr.Reader(['ar'])
    results = reader.readtext(img)
    texts = [text for bbox, text, prob in results]


    # 9. Clean result
    # text = text.strip()
    return texts

# Example usage:
digit = recognize_arabic_digit("letter_0.png", show_preprocessed=True)
print("Detected Arabic digit EASYOCR:", repr(digit))
# print("Detected Arabic digit:", repr(digit))


In [ ]:
import easyocr
import cv2
import numpy as np

reader = easyocr.Reader(['ar'], gpu=False)  # make this global or at module level

def prepare_for_easyocr_char(img):
    """
    img: numpy array (grayscale or RGB) containing ONE character.
    Returns a padded, upscaled image suitable for EasyOCR.
    """

    # If bool -> uint8
    if img.dtype == bool:
        img = (img * 255).astype(np.uint8)

    # Convert to grayscale
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        img = img.astype(np.uint8)

    # Normalize range
    if img.max() <= 1:
        img = (img * 255).astype(np.uint8)

    # Binarize a bit to stabilize
    _, img = cv2.threshold(img, 0, 255,
                           cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Make sure text is dark on light (helps EasyOCR)
    if img.mean() < 127:      # mostly dark => likely white on black
        img = 255 - img       # invert to black text on white

    # ---- FIX #1: add margin so glyph doesn't touch borders ----
    border = 20
    img = cv2.copyMakeBorder(
        img, border, border, border, border,
        borderType=cv2.BORDER_CONSTANT,
        value=255  # white background
    )

    # ---- FIX #2: upscale ----
    img = cv2.resize(img, None, fx=5, fy=5,
                     interpolation=cv2.INTER_LINEAR)

    return img


def recognize_single_char_easyocr(image_path, show=False):
    # load from file
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    prep = prepare_for_easyocr_char(img)

    if show:
        import matplotlib.pyplot as plt
        plt.imshow(prep, cmap='gray')
        plt.axis('off'); plt.show()

    # detail=0 -> just texts; paragraph=False so it doesn't merge things
    texts = reader.readtext(prep, detail=0, paragraph=False)

    return texts[0] if texts else ""

digit0 = recognize_single_char_easyocr("clean_digits.png", show=True)
letter0 = recognize_single_char_easyocr("clean_letters.png", show=True)
print("digit_0:", repr(digit0))
print("letter_0:", repr(letter0))


In [ ]:
'''
MAAW FORTNITE BATTLE ROYALE
'''

import numpy as np
import matplotlib.pyplot as plt
import cv2                         ### <<< you were using cv2 in save_characters

from skimage.io import imread
from skimage.color import rgb2gray, rgba2rgb
from skimage.filters import threshold_otsu
from skimage.morphology import (
    remove_small_holes,
    remove_small_objects,
    binary_closing,
    square,
)
from skimage.measure import label, regionprops
from skimage.transform import resize
from skimage.color import gray2rgb


# ---------------------------------------------------
# 1) SIMPLE GEOMETRIC UTILITIES
# ---------------------------------------------------

def crop_bottom(img, top_ratio=0.40):
    """
    Keep the lower part of the plate (digits + letters).
    top_ratio is the fraction of height we cut from the top.
    """
    h, w = img.shape[:2]
    return img[int(h * top_ratio):, :]


def split_digits_letters(bottom):
    """
    Egyptian plate: digits on the left half, letters on the right half.
    """
    h, w = bottom.shape[:2]
    mid = w // 2
    return bottom[:, :mid], bottom[:, mid:]


def get_mask(region):
    gray = rgb2gray(region)
    T = threshold_otsu(gray)
    mask = gray < T

    # ✅ KEEP holes & dots (light cleaning only)
    mask = remove_small_objects(mask, 25)
    return mask


# ---------------------------------------------------
# Helper: create a crop that is friendly for OCR
# ---------------------------------------------------
def make_ocr_char_crop(gray_sub, r0, c0, r1, c1,
                       pad=3, scale=3):
    """
    gray_sub : grayscale version of sub-image (0–1)
    (r0,c0,r1,c1) : bbox in sub coordinates
    pad : extra pixels around character
    scale : upscaling factor (so text is not tiny)
    """
    H, W = gray_sub.shape

    rr0 = max(r0 - pad, 0)
    cc0 = max(c0 - pad, 0)
    rr1 = min(r1 + pad, H)
    cc1 = min(c1 + pad, W)

    crop = gray_sub[rr0:rr1, cc0:cc1]        # 0–1 float from rgb2gray

    # upscale (don’t DOWNsize to 40×40)
    h, w = crop.shape
    crop = resize(crop, (int(h * scale), int(w * scale)),
                  anti_aliasing=True)

    return crop   # still float 0–1


# ---------------------------------------------------
# 2) DIGITS EXTRACTION
# ---------------------------------------------------

def touches_border(r0, c0, r1, c1, H, W, margin_ratio=0.005):
    mr = int(margin_ratio * H)
    mc = int(margin_ratio * W)

    if r0 <= mr: return True        # top
    if c0 <= mc: return True        # left
    if r1 >= H - mr: return True    # bottom
    if c1 >= W - mc: return True    # right

    return False


def extract_digits(digits_region, max_chars=4, debug=False):
    H0, W0 = digits_region.shape[:2]
    border_ratio = 0.04
    by = int(border_ratio * H0)
    bx = int(border_ratio * W0)
    sub = digits_region[by:H0 - by, bx:W0 - bx]

    # segmentation is done on mask
    mask = get_mask(sub)
    gray_sub = rgb2gray(sub)          ### <<< NEW: grayscale for OCR crops
    H, W = mask.shape

    lbl = label(mask)

    candidates = []
    for p in regionprops(lbl):
        r0, c0, r1, c1 = p.bbox

        touches = touches_border(r0, c0, r1, c1, H, W, margin_ratio=0.01)
        if touches:
            h = r1 - r0
            w = c1 - c0
            ar = p.area / (H * W)
            # ✅ only reject if it looks like a FRAME, not a digit
            if h > 0.9 * H or w < 0.02 * W or ar < 0.002:
                continue

        h = r1 - r0
        w = c1 - c0
        area = p.area
        wr = w / W
        ar = area / (H * W)

        if wr < 0.035 or wr > 0.75:
            continue
        if ar < 0.001:
            continue

        candidates.append({"bbox": [r0, c0, r1, c1], "area": area})

    # sort left → right
    candidates.sort(key=lambda c: c["bbox"][1])

    # merge overlapping boxes
    merged = []
    for c in candidates:
        if not merged:
            merged.append(c)
            continue

        r0, c0, r1, c1 = c["bbox"]
        r0m, c0m, r1m, c1m = merged[-1]["bbox"]
        overlap_x = min(c1, c1m) - max(c0, c0m)

        if overlap_x > 0:
            merged[-1]["bbox"] = [
                min(r0, r0m), min(c0, c0m),
                max(r1, r1m), max(c1, c1m)
            ]
        else:
            merged.append(c)

    boxed_mask = gray2rgb(mask.astype(np.uint8) * 255)

    chars = []
    for m in merged:
        r0, c0, r1, c1 = m["bbox"]
        if (r1 - r0) < 0.22 * H:
            continue

        # draw box for debug
        boxed_mask[r0:r1, [c0, c1-1]] = [255, 0, 0]
        boxed_mask[[r0, r1-1], c0:c1] = [255, 0, 0]

        # <<< OLD:
        # crop = resize(mask[r0:r1, c0:c1].astype(float), (40, 40))
        # chars.append((c0 + bx, crop))

        # >>> NEW: OCR-friendly grayscale crop with padding + upscaling
        crop = make_ocr_char_crop(gray_sub, r0, c0, r1, c1,
                                  pad=3, scale=3)
        chars.append((c0 + bx, crop))

    chars.sort(key=lambda x: x[0])

    if debug:
        return [c for _, c in chars][:max_chars], mask, boxed_mask
    else:
        return [c for _, c in chars][:max_chars]


# ---------------------------------------------------
# 3) LETTERS EXTRACTION
# ---------------------------------------------------

def extract_letters(letters_region, max_chars=3, debug=False):
    H0, W0 = letters_region.shape[:2]
    by = int(0.04 * H0)
    bx = int(0.04 * W0)
    sub = letters_region[by:H0 - by, bx:W0 - bx]

    mask = get_mask(sub)
    gray_sub = rgb2gray(sub)            ### <<< NEW: grayscale for OCR crops
    H, W = mask.shape
    lbl = label(mask)

    boxed_mask = gray2rgb(mask.astype(np.uint8) * 255)

    big_blobs = []   # main letter bodies
    small_blobs = [] # dots

    # -----------------------------
    # 1) SPLIT BIG BLOBS & DOTS
    # -----------------------------
    for p in regionprops(lbl):
        r0, c0, r1, c1 = p.bbox

        if touches_border(r0, c0, r1, c1, H, W, margin_ratio=0.02):
            continue

        h = r1 - r0
        w = c1 - c0
        area = p.area
        cy = p.centroid[0] / H
        wr = w / W
        ar = area / (H * W)

        # 👉 DOT = any small area < 0.015
        if ar < 0.015:
            small_blobs.append((r0, c0, r1, c1))
            continue

        # 👉 MAIN LETTER FILTER (no lower area limit now)
        if wr < 0.08 or wr > 0.75:
            continue
        if ar > 0.22:
            continue
        if cy > 0.75:
            continue

        big_blobs.append([r0, c0, r1, c1])

    # -----------------------------------------
    # 2) ATTACH EACH DOT TO NEAREST LETTER BOX
    # -----------------------------------------
    for dr0, dc0, dr1, dc1 in small_blobs:
        dc = (dc0 + dc1) / 2
        dr = (dr0 + dr1) / 2

        best_i = -1
        best_dx = 1e9

        for i, box in enumerate(big_blobs):
            r0, c0, r1, c1 = box
            bc = (c0 + c1) / 2
            br = (r0 + r1) / 2

            dx = abs(dc - bc)
            dy = abs(dr - br)

            # ✅ must be vertically close AND horizontally closest
            if dx < best_dx and dy < 120:
                best_dx = dx
                best_i = i

        # ✅ attach dot ONLY to the nearest valid letter
        if best_i != -1:
            box = big_blobs[best_i]
            box[0] = min(box[0], dr0)
            box[1] = min(box[1], dc0)
            box[2] = max(box[2], dr1)
            box[3] = max(box[3], dc1)

    # -----------------------------------------
    # 3) SORT & CROP FINAL LETTERS
    # -----------------------------------------
    big_blobs.sort(key=lambda b: b[1])

    chars = []
    for r0, c0, r1, c1 in big_blobs:
        boxed_mask[r0:r1, [c0, c1-1]] = [255, 0, 0]
        boxed_mask[[r0, r1-1], c0:c1] = [255, 0, 0]

        # <<< OLD:
        # crop = resize(mask[r0:r1, c0:c1].astype(float), (40, 40))
        # chars.append((c0 + bx, crop))

        # >>> NEW: grayscale + padding + upscaling
        crop = make_ocr_char_crop(gray_sub, r0, c0, r1, c1,
                                  pad=3, scale=3)
        chars.append((c0 + bx, crop))

    if debug:
        return [c for _, c in chars][:max_chars], mask, boxed_mask
    else:
        return [c for _, c in chars][:max_chars]


# ---------------------------------------------------
# 4) FULL PIPELINE FOR ONE IMAGE
# ---------------------------------------------------
def process_plate(img):
    bottom = crop_bottom(img)
    digits_region, letters_region = split_digits_letters(bottom)

    digits, d_mask, d_boxes = extract_digits(digits_region, debug=True)
    letters, l_mask, l_boxes = extract_letters(letters_region, debug=True)

    return digits_region, letters_region, digits, letters, d_mask, d_boxes, l_mask, l_boxes


# Choose your plate image
img22 = imread('Dataset/Limousines & tourist buses.png')

def save_characters(digits, letters):
    # digits / letters are floats 0–1 → uint8 0–255
    for i, img in enumerate(digits):
        out = (img * 255).astype(np.uint8)
        cv2.imwrite(f"digit_{i}.png", out)

    for i, img in enumerate(letters):
        out = (img * 255).astype(np.uint8)
        cv2.imwrite(f"letter_{i}.png", out)


d_reg, l_reg, digits, letters, d_mask, d_boxes, l_mask, l_boxes = process_plate(img22)

# (show_images assumed defined elsewhere)
show_images([d_reg, l_reg], ["Digits Region", "Letters Region"])
show_images([d_mask, l_mask], ["Digits Mask", "Letters Mask"])
show_images([d_boxes, l_boxes], ["Digits Mask + Boxes", "Letters Mask + Boxes"])

digit_titles = [f"Digit {i}" for i in range(len(digits))]
show_images(digits, digit_titles)

letter_titles = [f"Letter {i}" for i in range(len(letters))]
show_images(letters, letter_titles)

save_characters(digits, letters)
